In [ ]:
# import required packages/libraries
import pandas as pd
import numpy as np
import os

# Path to the data directory into which the cleaned data is saved.
csv_file_path = os.path.join("..", "EOC_Raw_Data", "EOC_Data_2017_TransCode.csv")
if not os.path.exists(csv_file_path):
    print("{} doesn't exist - perhaps the data cleaning script needs to be run?".format(csv_file_path))

# set option to display all columns in a dataframe 
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', None)

In [ ]:
# read data into dataframe
df = pd.read_csv(csv_file_path, quoting=2, parse_dates=True, infer_datetime_format=True, encoding="UTF-8",low_memory=False, skiprows=1)

# display first 5 rows in the dataframe
df.head()

In [ ]:
df.rename(columns={
'claimid':'Claim_ID',
'srvday':'Claim_Service_Day',
'rndrng prvdr':'Rendering_Provider',
'rndrng prvdrfullnme':'Rendering_Provider_Name',
'svc dprtmnt':'Service_Dept',
'appttype':'Appt_Type',
'patientid':'Patient_ID',
'ins pkg name':'Ins_Pkg_Name',
'primary clm ins pkg name':'Primary_Claim_Ins_Pkg_Name',
'secondary clm ins pkg name':'Secondary_Claim_Ins_Pkg_Name',
'proccode-descr':'Procedure_Codes_with_desc',
'proccode-grp procedure clas...':'Proc_Clas',    
'icd10transactiondiagcode1':'Trans10_Diagnosis_Code1',
'icd10transactiondiagcode2':'Trans10_Diagnosis_Code2',
'icd10transactiondiagcode3':'Trans10_Diagnosis_Code3',
'icd10transactiondiagcode4':'Trans10_Diagnosis_Code4',
'icd9transactiondiagcode1':'Trans9_Diagnosis_Code1',
'icd9transactiondiagcode2':'Trans9_Diagnosis_Code2',
'icd9transactiondiagcode3':'Trans9_Diagnosis_Code3',
'icd9transactiondiagcode4':'Trans9_Diagnosis_Code4',
'chg units sum':'Sum_Charge_Units',
'#chg':'Sum_Charges',
'net pmt':'Sum_Net_Payments',
'all chgs':'Sum_All_Charges',
'contract':'Sum_Contractual_Adj',
'allowed':'Sum_Actual_Allowed_Amts',
'expected':'Sum_Expected_Allowed_Amts',
'allow diff':'Sum_Allowable_Diff',
'pmt':'Sum_Payments',
'trnsfr type':'Transfer_Type',
'adjall':'Sum_All_Adj',
'current chg':'Current_Charge'
    },
    inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df['Transfer_Type'].unique()

################################################################################################################################
# Filter data only for Primary Payments
################################################################################################################################

In [ ]:
df['Ins_Pkg_Name'].unique()

In [ ]:
df = df.loc[df['Transfer_Type'] == 'Primary']
df = df.loc[(df['Primary_Claim_Ins_Pkg_Name'].str.startswith('Aetna')) | (df['Primary_Claim_Ins_Pkg_Name'].str.startswith('Meritain Health - Aetna')) |(df['Primary_Claim_Ins_Pkg_Name'].str.startswith('Chesterfield Resources - Aetna')) | (df['Primary_Claim_Ins_Pkg_Name'].str.startswith('1199SEIU Benefit and Pension Funds - Aetna'))]


In [ ]:
df.loc[df['Claim_ID'] == 981887]

In [ ]:
df.info()

In [ ]:
df['Proc_Code1'], df['Proc_Desc'] = df['Procedure_Codes_with_desc'].str.split(':', 1).str

In [ ]:
df['Proc_Code']= df['Proc_Code1'].str.split(',').str[0]

In [ ]:
df

In [ ]:
df['Proc_Code'].isnull().sum()

In [ ]:

df['Proc_Code'].unique()

In [ ]:
df['Proc_Desc'].unique()

In [ ]:
df

In [ ]:
df.Sum_All_Charges = df.Sum_All_Charges.astype(float).fillna(0.0)

In [ ]:
df.Sum_All_Adj = df.Sum_All_Adj.astype(float).fillna(0.0)
df.Sum_Contractual_Adj = df.Sum_Contractual_Adj.astype(float).fillna(0.0)
df.Sum_Charges = df.Sum_Charges.astype(float).fillna(0.0)
df.Sum_Net_Payments = df.Sum_Net_Payments.astype(float).fillna(0.0)
df.Sum_Actual_Allowed_Amts = df.Sum_Actual_Allowed_Amts.astype(float).fillna(0.0)
df.Sum_Expected_Allowed_Amts = df.Sum_Expected_Allowed_Amts.astype(float).fillna(0.0)
df.Sum_Allowable_Diff = df.Sum_Allowable_Diff.astype(float).fillna(0.0)
df.Sum_Payments = df.Sum_Payments.astype(float).fillna(0.0)

In [ ]:
df['Sum_Calculated_Allowable']= df['Sum_All_Charges'] - df['Sum_Contractual_Adj']

In [ ]:
df

In [ ]:
df['Trans10_Diagnosis_Code1'].unique()

In [ ]:
replacements = {
   'Trans10_Diagnosis_Code1': {
        
      
        r'(H3532.*)': 'AMD - Wet',
        r'(H35321.*)': 'AMD - Wet',
        r'(E10321.*)': 'Diabetes', 
        r'(E10331.*)': 'Diabetes', 
        r'(E10341.*)': 'Diabetes',
        r'(E10351.*)': 'Diabetes',
        r'(E11321.*)': 'Diabetes',
        r'(E11331.*)': 'Diabetes',
        r'(E11341.*)': 'Diabetes',
        r'(E11351.*)': 'Diabetes',
        r'(H3581.*)': 'ME',
        r'(H3483.*)': 'BRVO',
        r'(H3481.*)': 'CRVO',
       
   }
}
df.replace(replacements, regex=True, inplace=True)
df

In [ ]:
df['Trans10_Diagnosis_Code1'].unique()

In [ ]:
df['Trans10_Diagnosis_Code1'].replace(['H3570','H3023', 'H33102','H35371',
       'H35372', 'E113293', 'H33031', 'H43812', 'H35341', 'Z98890',
       'H35352', 'H3322', 'H43813', 'H35413', 'H35713', 'D3131', 'H33022',
       'H538', 'H33311', 'H43811', 'H209', 'H33012','H3509',
       'H3552', 'H4089', 'H353132', 'S0511XD', 'H35373', 'H44002',
       'H35053', 'H402222', 'H35351', 'E113593', 'H43823', 'H33021',
       'H33312', 'H4423', 'H4421', 'H1812', 'H18232', 'H3562', 'H33321',
       'H35052', 'H35721', 'H35343', 'Z961', 'H33011', 'E103592',
       'H401130', 'E113522', 'H35353', 'H35342', 'H33322', 'H4311',
       'H4312', 'H33302', 'H4419', 'H3530', 'H04123', 'E113532',
       'H353113', 'Z0000', 'H353111', 'H35359', 'H33013', 'H3341',
       'H33052', 'H3342', 'H35711', 'H31093', 'H353130', 'D333', 'H35021',
       'H40052', 'H40053', 'H26492', 'H1131', 'H3554', 'E119', 'R51',
       'H34232', 'H353114', 'H35712', 'H30133', 'H33301', 'H30142',
       'H33001', 'H35412', 'H353131', 'H33042', 'H4323', 'S0511XA',
       'D3132', 'E113591', 'E113393', 'E113391', 'H35051', 'H20013',
       'H33053', 'H04122', 'H40041', 'H353134', 'H35361', 'H43821',
       'G360', 'H47293', 'E113531', 'H44113', 'H35383', 'H43391',
       'H34823', 'E103293', 'H35363', 'H43392', 'H33193', 'H40013',
       'H35032', 'H33313', 'H4313', 'D1809', 'C6932', 'H35411', 'H3563',
       'H33051', 'H30141', 'Z79899', 'G43B0', 'H353122', 'H11221', 'H469',
       'H35463', 'H0016', 'H5319', 'H3411', 'H3553', 'H35443', 'H3093',
       'H43822', 'S0501XD', 'D571', 'E113291', 'H43393', 'H40022',
       'G43101', 'H16219', 'H33323', 'H34231', 'H16422', 'C6931',
       'H01009', 'H35042', 'H47323', 'H33303', 'H43399', 'H547', 'H53439',
       'D2310', 'H31011', 'E113592', 'Q120', 'H44001', 'H338', 'G932',
       'H26499', 'E113392', 'T66XXXA', 'T8522XD', 'H35723', 'Z9001',
       'H59022', 'E113521', 'Z973', 'H35103', 'H1820', 'H35033', 'H30131',
       'H35362', 'H35461', 'H211X3', 'H16002', 'H353112', 'H4062X0',
       'H3561', 'E103593', 'H20022', 'H25812', 'H01003', 'H20023',
       'H59021', 'H30043', 'H30891', 'H2513', 'H47013', 'H3412', 'H3582',
       'H04121', 'H20011', 'H468', 'H47092', 'H353121', 'H401224', 'Z794',
       'Q141', 'E103491', 'G43109', 'H35423', 'H31321', 'H31013',
       'H353124', 'H33192', 'H4063X0', 'H353133', 'H35022', 'H35073',
       'H25813', 'E113493', 'E103521', 'H2512', 'H35012', 'T85398A',
       'H31091', 'B0239', 'H4422', 'E103393', 'J300', 'E103292',
       'H401131', 'H16141', 'H259', 'H4601', 'H401133', 'H31002',
       'E113492', 'H3321', 'H21262', 'I10', 'H53423', 'H27111', 'B394',
       'S0501XA', 'H31092', 'Q140', 'H31411', 'H01023', 'H33101',
       'H33002', 'H1031', 'G43B1', 'B5801', 'H353120', 'G43901', 'H40059',
       'H40051', 'H3540', 'B830', 'H00039', 'H35071', 'H44112', 'H40043',
       'H47333', 'H401132', 'E1037X3', 'E113292', 'S0502XA', 'H462',
       'H3343', 'H3589', 'H4911', 'H27132', 'H35382', 'H18231', 'H409',
       'H35462', 'H3551', 'H20042', 'H40231', 'H43819', 'H318', 'H21542',
       'H1013', 'E1137X3', 'T66XXXD', 'H4010X3', 'H35041', 'H2589',
       'S0512XA', 'H35101', 'H0013', 'H47392', 'H31012', 'H26102',
       'H33023', 'H26493', 'H21301', 'H33191', 'H15102', 'H47292',
       'H5213', 'S0502XD', 'H15833', 'H53413', 'H33103', 'H26491',
       'H353110', 'H02412', 'H5500', 'H33003', 'H35722', 'H47091',
       'E103522', 'H21341', 'G453', 'H211X1', 'H47033', 'T1502XA',
       'H21561', 'H47011', 'H20012', 'H53123', 'M3210', 'H53421',
       'H30892', 'H5340', 'H40023', 'H53461', 'H578', 'Q131', 'H40011',
       'H2141', 'H5316', 'H11132', 'Q858', 'H31001', 'Q282', 'H3533',
       'H53431', 'Z83518', 'H16122', 'E113553', 'H53459', 'H25042',
       'H2702', 'H31422', 'H27122', 'H0520', 'H15111', 'S058X1A',
       'H44003', 'H44441', 'E1137X1', 'H44442', 'H01113', 'G500',
       'E103591', 'H524', 'H34211', 'H01116', 'H1032', 'S0512XD',
       'E1037X2', 'H1851', 'H15832', 'H4611','H44011', 'H401433',
       'Q142', 'T8529XA', 'H31103', 'H1033', 'H35469', 'H4710', 'D487',
       'H34212', 'H4921', 'H10013', 'H47291', 'G8918', 'H47321', 'H1132',
       'H4420', 'H5000', 'H01006', 'H401411', 'Z83511', 'D3192', 'E7871',
       'H401330', 'D3122', 'H53032', 'H401113', 'H27121', 'H532',
       'H35381', 'H11441', 'H30001', 'H21552', 'H53142', 'H5711', 'H3021',
       'G514', 'H47213', 'H44522', 'H27112', 'H44111', 'E103493',
       'E11319', 'S0592XA', 'D869', 'H16142', 'H04222', 'H5111', 'H33309',
       'H33032', 'I872', 'H5351', 'H4322', 'T8522XA', 'H35109', 'H16221',
       'H47012', 'E103392', 'E103531', 'H53462', 'H5712', 'H3323',
       'H4711', 'H3550', 'H30149', 'H40042', 'B488', 'E103291', 'H18613',
       'H25811', 'H27131', 'T8189XA', 'H442C2', 'H31301', 'H3091',
       'H18421', 'H35031', 'H47312', 'H442A1', 'A150', 'H33041', 'H25011',
       'H31311', 'H442A2', 'H4922', 'H00013', 'H442C3', 'H442A3', 'H349',
       'H1859', 'H5713', 'H3022', 'H442C1', 'B0052', 'H44021', 'H20032',
       'Z9841', 'H442D1', 'H31121', 'H4061X2', 'H548', 'Z947', 'H353194',
       'H35072', 'H4041X0', 'H442B1', 'H211X2', 'H33111', 'H53131',
       'Q8500','H35063', 'H4010X2',
       'H15033', 'H53432','H04552','H35433', 'H353123',
       'H3340', 'H53133', 'H40029', 'H401120', 'H35432','E113551',
       'E113552', 'G43801', 'S0502XS', 'E113491', 'H47311', 'H401122',
       'H25041', 'H527', 'H401112', 'H3523', 'H43312', 'H401123',
       'H31421', 'H2511', 'H30893', 'E109', 'H52213', 'H33332', 'H18603',
       'H43311', 'H33033', 'H21233', 'H4912', 'H25092', 'H31022',
       'H401124', 'H40021', 'H401134', 'H35431', 'H1133', 'H30143',
       'H47093', 'H44512', 'H4031X0', 'H401234', 'H16223', 'H4321',
       'H20051', 'G35', 'H40033', 'H35039', 'E11311', 'H11152', 'H2013',
       'H402210', 'H35011', 'S0512XS', 'H34822', 'H30113', 'H401110',
       'H40031', 'H401121', 'I63139', 'H21342', 'H16042', 'H4041X2',
       'E1137X2', 'D4981', 'H4043X0', 'H31322', 'H59023', 'H47233',
       'H21231', 'H47322', 'H30003', 'H2101', 'H4389', 'H21541', 'H31099',
       'H53411', 'H53451', 'H40019', 'H40012', 'H30033', 'E103391',
       'H15031', 'H15011', 'H35023', 'H25013', 'H21551', 'Z4801',
       'H35421', 'H33331', 'H30012', 'H442B2', 'H4301', 'D3162', 'H18612',
       'H16102', 'T1511XA', 'H20041', 'H02106', 'H53002', 'H30813',
       'H401190', 'H53003', 'H35171'],'OTHER',inplace=True)

In [ ]:
df['Trans10_Diagnosis_Code1'].unique()

In [ ]:
df['Trans10_Diagnosis_Code2'].unique()

In [ ]:
replacements = {
   'Trans10_Diagnosis_Code2': {
        
       
        r'(H3532.*)': 'AMD - Wet',
        r'(H35321.*)': 'AMD - Wet',
        r'(E10321.*)': 'Diabetes', 
        r'(E10331.*)': 'Diabetes', 
        r'(E10341.*)': 'Diabetes',
        r'(E10351.*)': 'Diabetes',
        r'(E11321.*)': 'Diabetes',
        r'(E11331.*)': 'Diabetes',
        r'(E11341.*)': 'Diabetes',
        r'(E11351.*)': 'Diabetes',
        r'(H3581.*)': 'ME',
        r'(H3483.*)': 'BRVO',
        r'(H3481.*)': 'CRVO',
       
 
   }
}
df.replace(replacements, regex=True, inplace=True)
df

In [ ]:
df['Trans10_Diagnosis_Code2'].unique()

In [ ]:
df['Trans10_Diagnosis_Code2'].replace(['H4313', 'H35063', 'H43812', 'H353112', 'H33012', 'E119',
       'H353131', 'H3561', 'H353111', 'H2513', 'H2511', 'Z794', 'H43811',
       'H35062', 'H31422', 'H5319', 'H35372', 'H43391', 'H40013',
       'H353121', 'Q141', 'H538', 'H43813', 'H16141', 'H43392', 'H4749',
       'H259', 'H353122', 'H35412', 'H4311',  'E113593',
       'Z961', 'H3582', 'H35373', 'H3322', 'H33321', 'D3132', 'Q131',
       'H33021', 'H33191', 'H35363', 'H33011', 'H35033', 'H35053',
       'H04552', 'E113291', 'H04122', 'H33022',
       'H3343', 'H4423', 'H35352', 'H43393', 'H5213', 'H35413', 'H35371',
       'H3562', 'H0016', 'H40051', 'H44442', 'E113592', 'H209',
       'H35722', 'H35721', 'H20011', 'H35031', 'H3509', 'E103292',
       'H3341', 'H353114', 'Z98890', 'H53002', 'H35411', 'W902XXA',
       'H16143', 'H40033', 'E113293', 'H353124', 'H35341', 'H4312',
       'H31091', 'H35351', 'H40041', 'H2512', 'H27112', 'H31012',
       'H35713', 'H3570', 'H25813', 'H26491', 'H47393', 'H35462',
       'E113391', 'D3131', 'H33322', 'H33051', 'H353123', 'H353130',
       'H353110', 'H532', 'E113292', 'H33302', 'H27132', 'H4010X1',
       'H3340', 'H59021', 'H211X1', 'B394', 'H2102', 'H31002', 'H35712',
       'H35461', 'H35732', 'H33312', 'H4089', 'H3022', 'H35711', 'H21543',
       'R51', 'E113591', 'H35023', 'H47293', 'H5347', 'H35463', 'H40023',
       'H30033', 'H3321', 'H35353', 'H26493', 'H3530', 'H33001', 'H35361',
       'H43821', 'H40053', 'H35071', 'E113492', 'E113392', 'H318',
       'H40052', 'H353132', 'H33192', 'H4321', 'H35432', 'S0502XD',
       'H44111', 'H33323', 'H33303', 'E113393', 'H47012', 'E113552',
       'H33311', 'H33301', 'H25811', 'H43823', 'H01003', 'E1137X1',
       'H3093', 'H35032', 'H401134', 'H35343', 'H401132', 'H43822',
       'H35342', 'H3554', 'H18232', 'S0502XS', 'H35103', 'H353120',
       'H35431', 'H43312', 'E113491', 'H47011', 'H16001', 'Q148',
       'H33102', 'H33329', 'H5703', 'H25041', 'H47029', 'H04129',
       'H33013', 'H43311', 'G43B0', 'H30132', 'B5801', 'H35362', 'H53413',
       'H3540', 'H3023', 'H2013', 'B20', 'H16142', 'H27111', 'H15833',
       'Q142', 'H3589', 'H35052', 'H33052', 'H33029', 'E113551', 'H47092',
       'C6930', 'H3533', 'H3523', 'E113521', 'H04123', 'H401131',
       'H35051', 'D2310', 'H15832', 'H4419', 'H26492', 'E103591',
       'E113532', 'H3323', 'H35042', 'H31423', 'H53133', 'H53431',
       'H20013', 'H47022', 'H3552', 'H01006', 'H4421', 'C6932', 'H1820',
       'H47233', 'H35073', 'H3342', 'H33103', 'H2522', 'S0501XA',
       'H53131', 'H33332', 'H44512', 'H33313', 'H40043', 'H5441',
       'H47091', 'G43B1', 'H4322', 'H53001', 'H33101', 'H16422', 'H40059',
       'H21233', 'H20012', 'H338', 'E103592', 'H5442', 'H1132', 'H31092',
       'H33053', 'H00013', 'H401133', 'I6529', 'H353133', 'H35433',
       'E1137X3', 'B029', 'H4912', 'H353113', 'H34231', 'E113553',
       'H31022', 'E109', 'H35359', 'H40042', 'H31093', 'H44002', 'H5010',
       'Z9001', 'H468', 'S0511XD', 'H3551', 'H40021', 'H547', 'H5000',
       'H31321', 'H3563', 'H268', 'H44112', 'H31011', 'H4420', 'H25812',
       'H30143', 'E103291', 'H5210', 'T8522XA', 'H4422', 'H401130',
       'H59022', 'Z83518', 'H40011', 'H40029', 'H2702', 'G43109', 'D1809',
       'H02403', 'H33193', 'H20051', 'I10', 'H33009', 'H401112',
       'H401234', 'E103593', 'G35', 'H409', 'H4063X0', 'H11442', 'H35039',
       'Q1389', 'H01113', 'T85398A', 'H25043', 'H35723', 'H35011',
       'H33031', 'H548', 'D3191', 'S0512XS', 'H33041', 'H34822', 'E0500',
       'H44001', 'H1045', 'H35022', 'Z79899', 'T85398D', 'H1131',
       'H02049', 'T8579XA', 'H479', 'H401110', 'H401122', 'H4711',
       'H47331', 'H34212', 'H4042X4', 'S0501XD', 'I63139', 'H20041',
       'H35043', 'H35021', 'H31411', 'H35443', 'H40031', 'H33002',
       'H5316', 'H25042', 'Z947', 'H35101', 'T66XXXA', 'H53123', 'H33032',
       'S0502XA', 'H21502', 'D869', 'H1859', 'H47323', 'H53452', 'H04121',
       'D3192', 'H18231', 'H4041X2', 'H4041X0', 'H21542', 'H401120',
       'H31301', 'H3411', 'H53022', 'H31311', 'S0512XA', 'E11311',
       'H43399', 'S058X2A', 'H401123', 'E1037X2', 'H26102', 'H02526',
       'H59023', 'H21342', 'H4720', 'H01116', 'H5711', 'E103522', 'H2701',
       'H30113', 'E1037X3', 'D4981', 'H44441', 'H20022', 'H1851',
       'H31322', 'E103293', 'H33003', 'E103491', 'H1812', 'H30813',
       'G453', 'S0512XD', 'H35419', 'H2589', 'Q8740', 'H30003', 'H2101',
       'H11152', 'H4389', 'H31013', 'H31421', 'H44011', 'H578', 'E103531',
       'Z7952', 'H35423', 'H53411', 'H40019', 'H4051X2', 'H33113',
       'H40003', 'H31103', 'H401331', 'Z9883', 'H5200', 'H179', 'H34239',
       'E103391', 'H15031', 'H18422', 'S0500XA', 'H524', 'H53421',
       'S0510XA', 'H40012', 'H35421', 'H401433', 'H33023', 'E113522',
       'Z952', 'H30001', 'H34211', 'H1811', 'K50919', 'S0511XA', 'M316',
       'H44421', 'H01026', 'G20', 'H4710', 'E113531', 'H27131', 'G459',
       'H442B2', 'H16222', 'T8189XD', 'H4301', 'H211X2', 'H25013',
       'H47292', 'H11001', 'D3162', 'H18892', 'G514', 'H53122', 'H401113',
       'H442A1', 'H44523', 'H20052', 'H442C1', 'S0501XS', 'G932',
       'G43909', 'H47032', 'H16221', 'H18011', 'H18612', 'H5712', 'M3210',
       'H16223', 'E1137X2', 'H16102', 'S0511XS', 'H11003', 'H02103',
       'H05241', 'H469', 'H3091', 'H40001', 'H30133', 'H11153', 'H53003',
       'Z9841', 'H16101', 'H4742','H15033',
       'H3522', 'H35072', 'H401111', 'G7000', 'H52213', 'H33033',
       'H401193', 'N189', 'H5202', 'H53423', 'H01023', 'H21551', 'H4921',
       'H2703', 'G43901', 'H462', 'H211X3', 'H27121', 'H34232', 'H3412',
       'H2510', 'H35383', 'H5371', 'H5501', 'C6931', 'B0239', 'T1502XA',
       'Z970', 'H30131', 'H47013', 'H02846', 'W19XXXA', 'H3553', 'L719',
       'H47392', 'H0520', 'H4323', 'S058X1A', 'H30142', 'H04223', 'Q282',
       'H353194', 'H35469', 'H47322', 'Z83511', 'D3122', 'H21552',
       'H53142', 'H15831', 'H5351', 'H44113', 'H30043', 'H53042',
       'H30149', 'H44021', 'H30012', 'H18613', 'H442C2', 'H18421',
       'H53031', 'H4922', 'H53433', 'H442A2', 'H401114', 'H442D1',
       'H20032', 'T1511XA', 'E113493', 'H442C3', 'Q871'],'OTHER',inplace=True)

In [ ]:
df['Trans10_Diagnosis_Code2'].unique()

In [ ]:
df['Trans10_Diagnosis_Code3'].unique()

In [ ]:
replacements = {
   'Trans10_Diagnosis_Code3': {
        
       
        r'(H3532.*)': 'AMD - Wet',
        r'(H35321.*)': 'AMD - Wet',
        r'(E10321.*)': 'Diabetes', 
        r'(E10331.*)': 'Diabetes', 
        r'(E10341.*)': 'Diabetes',
        r'(E10351.*)': 'Diabetes',
        r'(E11321.*)': 'Diabetes',
        r'(E11331.*)': 'Diabetes',
        r'(E11341.*)': 'Diabetes',
        r'(E11351.*)': 'Diabetes',
        r'(H3581.*)': 'ME',
        r'(H3483.*)': 'BRVO',
        r'(H3481.*)': 'CRVO',
       
   }
}
df.replace(replacements, regex=True, inplace=True)
df

In [ ]:
df['Trans10_Diagnosis_Code3'].unique()

In [ ]:
df['Trans10_Diagnosis_Code3'].replace(['Z794', 'H47321', 'H3562', 'H43811', 'H43812', 'H33303',
       'H4010X2', 'H401133', 'H401132', 'H33011', 'H15033',
       'H35431', 'E119', 'H4423', 'H35373', 'H43391', 'H401131', 'H43392',
       'H1131', 'H35412', 'H25043', 'H524', 'H25813', 'H43393', 'H5442',
       'H4311', 'H33302', 'H43813', 'H401130', 'H1859', 'H35343',
       'H40023', 'H538', 'H35413', 'H4322', 'H5213', 'H3561', 'T85398A',
       'H35371', 'H4312', 'Z98890', 'H2513', 'H16223', 'H409', 'E113292',
       'H259', 'H3554', 'H47091', 'H35362', 'H35033',
       'H35351', 'E113591', 'H4922', 'Z961', 'H35372', 'Z79899',
       'H353111', 'H3530', 'H26013', 'H26491', 'H40049',
       'H25811', 'E113293', 'H33311', 'H04121', 'H401193', 'H25812',
       'H3522', 'H33012', 'H35352', 'H35342', 'H1820', 'H2511', 'H4321',
       'Q141', 'H353110', 'H26492', 'H3323', 'H353131', 'H35433',
       'H35722', 'H25013', 'H2589', 'H2512', 'H3523', 'H2102', 'H35361',
       'I10', 'H01003', 'H35462', 'S0501XA', 'H04123', 'H353112',
       'H33321', 'H353120', 'H353124', 'E113392', 'H532', 'E113593',
       'H35421', 'H40029', 'H50112', 'H4089', 'H33021',
       'H4313', 'D3132', 'H31092', 'H5319', 'H5203', 'H35363', 'H401120',
       'H35411', 'E0500', 'H35463', 'H26493', 'D3131', 'E113551', 'H5316',
       'H43822', 'H40053', 'H209', 'H35341', 'H44113', 'H40013', 'H3563',
       'H401123', 'D3122', 'H35353', 'H353121', 'H548', 'H30132',
       'H353130', 'H47022', 'G43B0', 'H353113', 'H34212', 'H04129',
       'H40051', 'H53031', 'H401113', 'Z947', 'H33023', 'H47311',
       'H35461', 'H31093', 'H47012', 'H3509', 'H353123', 'H35432',
       'H33022', 'H53002', 'M3210', 'H3570', 'E113291', 'H527', 'E70329',
       'H34211', 'H11001', 'H53123', 'E113592', 'H4061X0', 'B5801',
       'Z9001', 'H34231', 'H59022', 'H43823', 'H401112', 'E1137X1',
       'H3589', 'H33301', 'Q142', 'H31091', 'H2022', 'H35423', 'H33322',
       'H353122', 'H35359', 'S0511XA', 'H33312', 'H40033', 'S0501XD',
       'H5441', 'H33103', 'H16042', 'H31103', 'G43101', 'I872', 'H4923',
       'L719', 'E103593', 'H43311', 'H52213', 'H33003', 'S0552XS',
       'H3411', 'H33051', 'H33032', 'H3342', 'H179', 'H47231', 'E113521',
       'S0512XD', 'H401122', 'H33002', 'S0231XA', 'H5000', 'H35013',
       'H353132', 'H33193', 'H1132', 'H2013', 'H16141', 'E113532',
       'H02402', 'H53001', 'H34232', 'H04223', 'H25092', 'H33102',
       'H34213', 'N189', 'H31003', 'E1137X3', 'H5202', 'H353133',
       'H35032', 'H33192', 'H401124', 'H40011', 'H211X3', 'H33101',
       'A6920', 'H21551', 'H4921', 'H1133', 'E113391', 'H2703', 'H401331',
       'H3341', 'H52203', 'G510', 'B20', 'H20011', 'H00023', 'H35712',
       'H04122', 'H401121', 'H25819', 'H4419', 'G453', 'H40041', 'E11311',
       'H35052', 'H11152', 'H01116', 'E113552', 'H33053', 'H4062X0',
       'H40021', 'H3321', 'H33112', 'H5210', 'H2510', 'H35029', 'E113491',
       'H33001', 'H31411', 'H35051', 'H35713', 'H35041', 'H3533',
       'H01006', 'H53133', 'H18411', 'H27111', 'H40019', 'H5501',
       'H21233', 'E103591', 'E113553', 'H30113', 'B0239', 'E113393',
       'H3540', 'H2701', 'Z970', 'Z8571', 'G932', 'H3551', 'I679',
       'H35031', 'H539', 'H25012', 'H21541', 'H40052', 'H33319', 'H211X1',
       'H18232', 'C6931', 'H02846', 'H35721', 'H25042', 'H35101',
       'H21542', 'G43109', 'H21342', 'C6930', 'H47323', 'H27112',
       'E103291', 'H11442', 'H2702', 'H4422', 'H1851', 'H3582', 'H25031',
       'H338', 'H401110', 'I252', 'T8522XA', 'H35062', 'H33052',
       'S0511XD', 'S0502XA', 'H10022', 'H33313', 'H353114', 'H35383',
       'S0512XS', 'H43821', 'H31301', 'H35723', 'H35053', 'H53143',
       'H25093', 'H5702', 'H20013', 'H3322', 'H4043X0', 'H318', 'H30143',
       'H47322', 'H35021', 'H40042', 'S0512XA', 'H53021', 'H401234',
       'H47233', 'H35073', 'H353194', 'H31322', 'H2101', 'H47092',
       'H4323', 'H02103', 'H578', 'H44011', 'H15833', 'E113492', 'H31099',
       'E103592', 'H31002', 'Z9883', 'D3192', 'H53451', 'H31011',
       'H40012', 'H30033', 'H35043', 'H5052', 'H31421', 'D571', 'H401134',
       'H2640', 'H26012', 'H18231', 'H11153', 'I6529', 'H43819', 'H15011',
       'H33323', 'G35', 'H0013', 'S0230XA', 'H44111', 'G514', 'H44512',
       'H16143', 'H401433', 'E109', 'H27132', 'H5010', 'H27131', 'H16142',
       'H35071', 'H35072', 'H30133', 'H35443', 'H30012', 'H15031',
       'H53042', 'H35061', 'H33329', 'H26412', 'H33332', 'H3343',
       'H43399', 'H30149', 'H33013', 'H44421', 'H59021', 'H53433',
       'H3412', 'H30142', 'H4421', 'H2011', 'H401114', 'H1811', 'G4733',
       'H442C2', 'H47292', 'D499', 'S0502XD', 'H44002', 'M352', 'S0511XS',
       'H18422', 'H44441', 'H02106', 'D6959', 'E113531', 'H4601',
       'E113493', 'H442C1', 'H31001', 'H31312',
       'H4389', 'H35059', 'H1089', 'H21232', 'H15832', 'H15831',
       'H4031X0', 'H5201', 'H40022', 'H27121', 'T85398D', 'B0233',
       'H30141', 'H27122', 'W19XXXA', 'Y92129', 'H53032', 'S058X2A',
       'Y92128', 'H59023', 'D3191', 'H35732', 'E113522', 'Z4801', 'H3023',
       'H44112', 'H5711', 'E1137X2', 'H5359', 'H47392', 'H44442',
       'H47011', 'H31422', 'H20032', 'Q148', 'H4912', 'H3552', 'H353134','H4749'],'OTHER',inplace=True)

In [ ]:
df['Trans10_Diagnosis_Code3'].unique()

In [ ]:
df['Trans10_Diagnosis_Code4'].unique()

In [ ]:
replacements = {
   'Trans10_Diagnosis_Code4': {
        
       
        r'(H3532.*)': 'AMD - Wet',
        r'(H35321.*)': 'AMD - Wet',
        r'(E10321.*)': 'Diabetes', 
        r'(E10331.*)': 'Diabetes', 
        r'(E10341.*)': 'Diabetes',
        r'(E10351.*)': 'Diabetes',
        r'(E11321.*)': 'Diabetes',
        r'(E11331.*)': 'Diabetes',
        r'(E11341.*)': 'Diabetes',
        r'(E11351.*)': 'Diabetes',
        r'(H3581.*)': 'ME',
        r'(H3483.*)': 'BRVO',
        r'(H3481.*)': 'CRVO',
       

   }
}
df.replace(replacements, regex=True, inplace=True)
df

In [ ]:
df['Trans10_Diagnosis_Code4'].unique()

In [ ]:
df['Trans10_Diagnosis_Code4'].replace([ 'H43392', 'H44433','G43101', 'H25013', 'H43812',
       'H259', 'H2513', 'H33321', 'Z98890', 'H53432', 'H43813', 'H353130',
       'Z961', 'E119', 'H33103', 'G514', 'H44521', 'H43393', 'H3561',
       'H35413', 'H25813', 'H409', 'H4312', 'H33301', 'H04123', 'H35371',
       'H35033', 'Z794', 'H43391', 'M3210', 'H33021', 'H353121', 'H5316',
       'H401133', 'H33001', 'H25811', 'H35103', 'D3132', 'H35433',
       'H40051', 'H401113', 'H33052', 'H3562', 'H2512', 'H401132',
       'H4311', 'H35721', 'H1859', 'Z7952', 'E113592', 'I10', 'H33311',
       'H43811', 'E113591', 'T85311A', 'H26493', 'H01006', 'H35031',
       'H26492', 'H538', 'H35722', 'H35363', 'E113493', 'H26491',
       'H35372','H5359', 'H1132', 'H0013', 'H4322', 'H43821',
       'H40013', 'H3523', 'H35412', 'H5213', 'H33322', 'H35351', 'H01003',
       'H4010X2', 'H35053', 'H53133', 'H3582', 'H35361', 'H4423',
       'H30133', 'D3131', 'H2511', 'H25093', 'H1840', 'E113292',
       'H401131', 'H4389', 'H33313', 'D869', 'H401130', 'H33312',
       'G43801', 'H353122', 'H35431', 'H35059', 'H33101', 'H35411',
       'H4420', 'H4089', 'H35461','D3192', 'Z79899', 'H209',
       'H401122', 'H2589', 'H35362', 'H35051', 'S0512XS', 'H31091',
       'H16223', 'H35072', 'H1089', 'H53002', 'H353110', 'H31093',
       'H40043', 'H33303', 'H02402', 'H33323', 'H25812', 'H35373',
       'H35443', 'H353112', 'H15833', 'H3530', 'H401112', 'H401123',
       'H3551', 'H53121', 'H2701', 'H401111', 'H3321', 'H353111',
       'Q141', 'H40053', 'H179', 'H44529', 'H40023', 'G7000', 'H30043',
       'H35432', 'H338', 'H40042', 'H4313', 'H33012', 'H40033', 'S0522XS',
       'E782', 'H2702', 'E113593', 'H3540', 'H4321', 'H15831',
       'H18603', 'H31011', 'Z947', 'H5201', 'H3322', 'H33033', 'H35713',
       'H35462', 'H21232', 'E113293', 'H25819', 'H33011', 'E109',
       'H353124', 'H53032', 'H35032', 'H1851', 'H5441', 'E103593',
       'H53001', 'H44511', 'H35423', 'H53423', 'B20', 'H1820', 'H11002',
       'H35352', 'H34231', 'H02103', 'H43822', 'H2101', 'H401134',
       'H35463', 'M0800', 'E113291', 'H35342', 'H25042', 'H47093',
       'H4031X0', 'H532', 'E103591', 'H3342', 'H1811', 'H2703', 'H47012',
       'H34212', 'H35052', 'E113492', 'H3589', 'H35343', 'E103592',
       'H35021', 'E70329', 'H3343', 'H349', 'C6932', 'G43B0', 'H40022',
       'H35711', 'H33111', 'H30132', 'H5442', 'H27121', 'H35041',
       'H33002', 'H402210', 'H15001', 'E1137X3', 'H33302', 'H35353',
       'H5371', 'H21342', 'H40049', 'H35341', 'H11442', 'H353114',
       'H4901', 'H33051', 'H44002', 'B0233', 'E113392', 'Z4801', 'H31092',
       'H18422', 'H27122', 'H16423', 'H40012', 'E113491', 'H318',
       'W19XXXA', 'H33023', 'S0511XS', 'H524', 'S0502XD', 'H01113',
       'H04121', 'H211X1', 'S0511XA', 'G35', 'H16042', 'H353113', 'G510',
       'H21233', 'H539', 'S0501XD', 'I4891', 'E113521', 'H211X3',
       'S0502XA', 'H18413', 'S0512XA', 'H47011', 'H47323', 'S0511XD',
       'H47091', 'H5712', 'H4062X0', 'H4323', 'H353131', 'Y92128',
       'H31411', 'H3092', 'H547', 'H2640', 'H40011', 'E113391', 'H02403',
       'H11153', 'S0501XA', 'H18232', 'H5319', 'H31002', 'H3509',
       'H33192', 'H40041', 'H21231', 'H53031', 'H3554', 'S0512XD',
       'B0239', 'H04129', 'H40213', 'H4422', 'H353132', 'H5711', 'H00023',
       'H4922', 'H4061X2', 'H33022', 'H47322', 'H47092', 'H4421', 'H5203',
       'E1137X1', 'H15832', 'H21541', 'H31103', 'H33193', 'H40019',
       'H18423', 'H33331', 'H3411', 'H04209', 'H44021', 'H31099', 'H3093',
       'E113551', 'H21551', 'H33013', 'H59021', 'H0233', 'D499', 'H21542',
       'H33332', 'H34232', 'H30131', 'H16143', 'H31012', 'H31421',
       'H35042', 'H5210', 'H16101', 'H3563', 'H4720', 'D649', 'H401211',
       'N189', 'B029', 'T8522XA', 'H43823', 'H35023', 'Q142', 'H401110',
       'H43313', 'H27111', 'H04122', 'H44442', 'H402233', 'H35712',
       'H33102', 'E113552', 'H20013', 'H353133', 'H18061', 'H59022',
       'H4912', 'H35359', 'H5211', 'S0231XA', 'H1131','H4749','H5010',
       'E113553', 'H35731', 'Y92129', 'E11319', 'H18231',
       'H18613', 'Q871'],'OTHER',inplace=True)

In [ ]:
df['Trans10_Diagnosis_Code4'].unique()

In [ ]:
replacements = {
   'Trans9_Diagnosis_Code1': {
        
       
        r'(36250.*)': 'AMD - Wet',
        r'(36252.*)': 'AMD - Wet',
        r'(36283.*)': 'ME',
        r'(36235.*)':'CRVO',
        r'(36236.*)':'BRVO',
        r'(36.207*)':'Diabetes'
       
   }
}
df.replace(replacements, regex=True, inplace=True)
df

In [ ]:
df['Trans9_Diagnosis_Code1'].unique()

In [ ]:
df['Trans9_Diagnosis_Code1'].replace(['V5867', '37923', '36321', '36218', '36110','37921',
       '36256', '36101', '25000', '36510', '36103', '36281', '36033',
       '36254', '36253','36371', '3612', '37907', '36263', '36501',
       '36241', '36216', '2246', '74356', '36102', '37021', '3688',
       '36843', '36132', '36619', '3643', '37924', '36217', '36274',
       'V431', '36589', '36284', '36505', '36610', 'V5889', '36000',
       '36523', '37927', '36334', '36257', '36211', '36021', '37122',
       '36131', '3659', '36181', '36242', '36276', '36130', '3671',
       '3732', '36031', '37854', 'V5869', '37922', '36401', '36019',
       '37515', '36616', '36502', 'V700', '36261', '36107',
       '11500', '2251', '36332', '37152', '36212', '36504', '36653',
       '37272', '3779', '2249', '37300', '36232', '36313', '36617',
       '36315', '36100', '36442', '4019', '36104', '9181', '9212',
       '36243', '36503', '36330', '3410', '37715', '36266', '36012',
       '36304', '36255', '36237', '36215', '36119', '37934', '22809',
       '1906', '37741', '36961', '34620', '37810', '36815', '36816',
       '37261', '37730', '36231', '36275', '36264', '28260',
       '9062', '34602', '36801', '37034', '37000', '37062', '36214',
       '37721', '37925', '3699', '3679', '2161', '3682', '1302', '74330',
       '36189', '3482', '99882', '990', '37932', '37912', '36289',
       'V4578', '36343', '36229', 'V410', '37120', '36531', '36402',
       '35800', '36340', '37749', '25001', '36618', '36811', '36133',
       '36512', '36042', '34600', '36262', '36363', '36800', '36453',
       '36213', '99659', '37033', '43310', '05329', '37853', '4779',
       '36331', '37731', '36842', '74351', '36372', '37302', '08881',
       '3668', '37200', '34621', '34692', '99653', '1280', '37313',
       '36233', '37430', '37724', '37733', '37275', '25050', '36603',
       '74349', '36273', '36410', '36404', '36521', '3638', '36471',
       '37205', '36614', '37722', '36460', '37900', '37214', '3518',
       '36841', '37403', '37433', '37950', '36463', '36234', '36311',
       '37743', '9300', '36475', '7100', '37700', '43311', '37100',
       '36840', '36846', '36306', '37993', '74345', '36474', '36106',
       '37255', '7596', '74781', '135', '340', '37001', '74357', 'V1919',
       '37023', '412', '36844', '37931', '36361', '37933', '37630',
       '37203', '37901', '8719', '36362', '37332', '3501', '37991',
       '75989', '3674', '36562', '37157', '37714', '36322', '37732',
       'V4569', '36202', '36002', '36552', '75982', '36441', '37410',
       '37929', '37701', '2388', '37202', '33818', '37800', '36513',
       '2245', '3670', '36300', '36477', '36813', '37903', '37711',
       '36041', '37522', '37883', '3694', 'V5831', '45981', '36854',
       '36305', '118', '37161', '99889', '3320', '37143', '37723',
       '01190', '36615', '37020', '37311', '05443', '36004', '36403',
       'V4561', '36277', '36001', 'V425', '9063', '9301', '23770',
       '24200'],'OTHER',inplace=True)

In [ ]:
df['Trans9_Diagnosis_Code1'].unique()

In [ ]:
df['Trans9_Diagnosis_Code2'].unique()

In [ ]:
replacements = {
   'Trans9_Diagnosis_Code2': {
        
       
        r'(36250.*)': 'AMD - Wet',
        r'(36252.*)': 'AMD - Wet',
        r'(36283.*)': 'ME',
        r'(36235.*)':'CRVO',
        r'(36236.*)':'BRVO',
        r'(36.207*)':'Diabetes'
       
   }
}
df.replace(replacements, regex=True, inplace=True)
df

In [ ]:
df['Trans9_Diagnosis_Code2'].unique()

In [ ]:
df['Trans9_Diagnosis_Code2'].replace(['V5867', '36218', '37921', '36101', '25000', '36281', '36616',
       '36371', '36815', '36256', '37924', '36501', '74356', '3688',
       '37272', '37751', '36610', '36263', '36961', '37923', 'V431',
       '37152', '36284', '3612', '36131', '37922', '3671', '74345',
       '36102', '36119', '37033', '36216', '37556', '37123', '3659',
       '36021', '36619', '36504', '36211', '3643', '36253',
       '36217', '36100', '36800', '36603', '37021', '36653', '36276',
       '36254', '36503', '36505', '36257', '37932', '36242',
       '36241', '36240', '36261', '36266', '2246', '36615', '3682',
       '36130', '37934', '36441', '36181', '99882', '11500', '36330',
       '37515', '36132', '36321', '36471', '7840', '36212', '37715',
       '36847', '36107', '36589', '36304', '37120', '36334', '37927',
       '3638', '37300', 'V5889', '36012', '2245', '3694', '36612',
       '37742', '36233', '37122', '9062', '37925', '37741',
       '37000', '74359', '36110', '37942', '36614', '34620', '2702',
       '36313', '37240', '1302', '36841', '36531', '36260', '36410',
       '042', '4019', '37912', '74357', '36289', '37749', '1906', '36343',
       '36229', '36214', '36332', '36570', '36843', '36401', '36502',
       '37714', '3699', '45981', '37854', '9060', '36231', '36133',
       '37100', '9181', '2859', '36232', '34621', '37800', '37062',
       '36453', '36189', '37430', '37311', '37522', '0539', '5859',
       '36331', '25001', '37810', '36451', '36442', '36273', '36363',
       '36215', 'V5869', '36315', '99653', 'V1919', '37931', '3689',
       '34600', '22809', '36405', '36512', '340', '25050', '37332',
       '36274', '99659', '36000', '36213', '36103', '2249', '9063',
       '36237', '24200', '7102', '37141', '99669', '36019', '3779',
       'V430', 'V5831', '37701', '37724', '43311', '36404', '37143',
       '36372', '36264', '36340', 'V425', '990', '36812', '36470',
       '36844', '37275', '412', '36562', '36361', '9212', '36201',
       '36362', '36811', '37446', '36463', '37710', '37991', '36813',
       '37941', '36311', '36031', '05329', '36803', '36322', '36234',
       '36801', '36617', '75982', '36816', '37251', '37993', '37929',
       '36002', 'V5865', 'V4569', '36262', '36560', '36500', '36513',
       '37907', '3674', '3732', '36842', '36042', '36552', 'V433',
       '36300', '37520', '5559', '37157', '4465', '36032', '37700',
       '4359', '4379', '37926', '37189', '3518', '36041', '3482', '36207',
       '34690', '135', '37111', '37161', '2241', '7100', '37020', '37410',
       '36650', '37631', '37853', 'V4511', '37730', '37721', '37752', '36370','35800',
       '6953', '36722', '36574', '3670', '37302', '34692', '36230',
       '37733', '37933', '36255', 'E914', '9300', '37482', 'E8889',
       '36275', '36840', '42731', '37722', '37630', '8719', 'E8497',
       '3501', '74781', '36573', '36571', 'V1911', '36243', '36477',
       '36306', '28260', '36004', '36305', '36104', '36403', '9301'],'OTHER',inplace=True)

In [ ]:
df['Trans9_Diagnosis_Code2'].unique()

In [ ]:
df['Trans9_Diagnosis_Code3'].unique()

In [ ]:
replacements = {
   'Trans9_Diagnosis_Code3': {
        
       
        r'(36250.*)': 'AMD - Wet',
        r'(36252.*)': 'AMD - Wet',
        r'(36283.*)': 'ME',
        r'(36235.*)':'CRVO',
        r'(36236.*)':'BRVO',
        r'(36.207*)':'Diabetes'
       
   }
}
df.replace(replacements, regex=True, inplace=True)
df

In [ ]:
df['Trans9_Diagnosis_Code3'].unique()

In [ ]:
df['Trans9_Diagnosis_Code3'].replace(['37721', '36281', '37921', '36130', '36510', '37300',
       '37907', '36261', '25000', '36021', '36256', '37924', '36610',
       '36616', '36263', '36614', '3674', 'V5867', '36619', '37923',
       '36254', '36505', '3688', '99659', '37515', '37749', '36257',
       '36253', '3659', 'V431', 'V5869', '36102', '36816', '36503',
       '36132', '36101', '37120', '37152', '74356', '3612', '36617',
       '36229', '36615', '36211', '9181', '36504', '36131', '99882',
       '3682', '36262', '37811', '37272', '3732', '36110', '37922',
       '36815', '3670', '24200', '36266', '3671', '36441', '36334',
       '4019', '36012', '36501', '36612', '37140', '37929', '37927',
       '34620', '36589', 'V425', '37723', '36800', '7100', '9063',
       '36240', '37033', '08881', '3679', '37239', '36233', '2246',
       '37240', 'V4578', '37430', '36289', '74357', '36423',
       '36812', '37912', '9212', '37100', '36041', '35800', '36189',
       '37001', '34602', '36100', '37925', '36722', '2722', '36103',
       '36574', 'V5889', '36340', '36213', '36119', '36410', '37021',
       '36276', '36232', '36801', '37157', '36961', '36313', '36330',
       '36042', '37410', '36477', '36241', '37931', '36513', '36653',
       '3510', '36565', '37312', '36181', '37741', '36216',
       '36212', '2702', '3643', '36234', '36201', '36107', '36523',
       '36372', '36214', '3518', '3694', '37951', '36453', '37275',
       '36311', '05329', '36260', 'V1072', '1302', '3482', '36273',
       '4379', '37062', '36442', '37122', '1906', '37482', '36242',
       '34600', '36463', '37932', '36284', '36613', '42731', '36570',
       '36019', '36231', '36218', '36803', '36207', '36361', '36531',
       'E8497', '36255', '36471', '36401', '36315', '2249', '36512',
       '37714', '25001', '36363', '36300', '36002', '25050', '36844',
       '36304', '37842', '36371', '28260', '36650', '37903', '340',
       '37143', '36552', '36573', '37810', '37934', '36502', '36230',
       '36264', '36305', '37434', '2399', '36133', '36332', '36032',
       '36843', '2859', '5859', '32723', '1361', '36031', '28749',
       '37731', 'V4569', '36362','37751', '042',
       '36321', '37933', '37900', '37701', 'E8889', '3699', '22809',
       '36217', '3638', '11500', '37991', '36000', '36601', '37722',
       '135', '36403', '74359'],'OTHER',inplace=True)

In [ ]:
df['Trans9_Diagnosis_Code3'].unique()

In [ ]:
df['Trans9_Diagnosis_Code4'].unique()

In [ ]:
replacements = {
   'Trans9_Diagnosis_Code4': {
        
       
        r'(36250.*)': 'AMD - Wet',
        r'(36252.*)': 'AMD - Wet',
        r'(36283.*)': 'ME',
        r'(36235.*)':'CRVO',
        r'(36236.*)':'BRVO',
        r'(36.207*)':'Diabetes'
       
   }
}
df.replace(replacements, regex=True, inplace=True)
df

In [ ]:
df['Trans9_Diagnosis_Code4'].unique()

In [ ]:
df['Trans9_Diagnosis_Code4'].replace(['37924', '34602', '36615', '37921', '36616', '36131', '36843',
       'V431', '25000', '36110', '3518', '36041', '36334', '37515',
       '3659', '36263', '36619', 'V5867', '36281', '36130', '36211',
       '7100', '36610', '2246', '36261', '36107', '37923',
       '36256', 'V5865', '4019', '36132', '36653', '36501','36859',
       '36229', '3671', '36253', '36257', '36021', '36313', '135',
       '37300', '36216', '36266', 'V5869', '36215', '37810', '36570',
       '36503', '36264', '37152', '37912', '36273', '36589', '37931',
       '3688', '36505', '6953', '25001', '36502', '36260', '36102',
       '37160', '36332','3670', '36504', '37033', '3612',
       '36103', '36241', '36453', '36101', '74356', '36254', '36242',
       '37157', '36800', '36262', '36842', '042', '37120', '36232',
       '37927', '36441', '3643', '71430', '36571', '37749', '36401',
       '37123', '36289', '36574', '36100', '36230', '1906', '34620',
       '36207', '37933', '7102', '36214', '37900', '36463', '37851',
       '05321', '37741', 'E8889', '9063', '3674', '36189', '37332',
       '36442', '340', '3510', '3689', '9181', '37721', '37100', '37991',
       '36372', '36612', '3699', '36523', '36650', '37251', '37122',
       '36815', '36330', '36119', 'V4569', '36573', '36522', '37312',
       '3682', '36255', '2249', '36471', '36340', '37141', '36181',
       '36133', 'V5889', '36231', '36004', '36477', '99882', '36601',
       '37021', '36371', '37020', '36512', '0539', '36212', '74357',
       '37925', '37932', '37715', '36801', '37112', '36233', '36617',
       '37272','37751', 'E914',
       'E8497', '36201', '37161', '24200'],'OTHER',inplace=True)

In [ ]:
df['Trans9_Diagnosis_Code4'].unique()

In [ ]:
df.head()

In [ ]:
df['Proc_Code'].isnull().sum()

In [ ]:
df.info()

In [ ]:
df2 = pd.DataFrame({'Claim_ID':df.Claim_ID.unique()})
df2['Proc_Clas'] = [list(set(df['Proc_Clas'].loc[df['Claim_ID'] == x['Claim_ID']])) for _, x in df2.iterrows()]

In [ ]:
df2

In [ ]:
df2.to_csv('df2.csv', index=False)

In [ ]:
df2.info()

In [ ]:
df2['Proc_Clas']=df2['Proc_Clas'].astype(str)

In [ ]:
df2['Proc_Clas1'], df2['Proc_Clas2'] = df2['Proc_Clas'].str.split(',', 1).str

In [ ]:
df2

In [ ]:
df2.isnull().sum()

In [ ]:
df2 = df2[pd.notnull(df2['Proc_Clas2'])]

In [ ]:
df2

In [ ]:
df_Modifier = df[df['Claim_ID'].isin(df2['Claim_ID'])]

In [ ]:
df_Modifier 

In [ ]:
df_Lumped_NonDrug = df[~df['Claim_ID'].isin(df2['Claim_ID'])]

In [ ]:
df_Lumped_NonDrug

In [ ]:
df_Modifier.loc[df_Modifier['Claim_ID'] == 1077095]

In [ ]:
df_Modifier.info()

In [ ]:
#Function to create output data folder to store graph and .csv 
def create_directory_for_output():
    data_folder = os.path.join("..", "EOC_Raw_Data", 'EOC_Reports_All_Insurance')
    if not os.path.exists(data_folder):
        os.mkdir(data_folder)
    return data_folder

output_folder=create_directory_for_output()
output_folder

################################################################################################################################
# AMD - Wet
################################################################################################################################

In [ ]:
df_AMD_Wet=df_Modifier.loc[(df_Modifier['Trans10_Diagnosis_Code1'] == 'AMD - Wet') 
                           |(df_Modifier['Trans10_Diagnosis_Code2'] == 'AMD - Wet')
                           |(df_Modifier['Trans10_Diagnosis_Code3'] == 'AMD - Wet')
                           |(df_Modifier['Trans10_Diagnosis_Code4'] == 'AMD - Wet') 
                           |(df_Modifier['Trans9_Diagnosis_Code1'] == 'AMD - Wet') 
                           |(df_Modifier['Trans9_Diagnosis_Code2'] == 'AMD - Wet')
                           |(df_Modifier['Trans9_Diagnosis_Code3'] == 'AMD - Wet')
                           |(df_Modifier['Trans9_Diagnosis_Code4'] == 'AMD - Wet')]
                          

In [ ]:
df_AMD_Wet= (df_AMD_Wet.groupby(['Claim_ID','Proc_Clas','Proc_Code','Proc_Desc'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index()
                 )

In [ ]:
df_AMD_Wet

In [ ]:
df_AMD_Wet.to_csv('../EOC_Raw_Data/df_amd_wet.csv')

In [ ]:
df_AMD_Wet_DRUG=df_AMD_Wet[df_AMD_Wet['Proc_Clas']=='Drug']

In [ ]:
df_AMD_Wet_DRUG

In [ ]:
#df_AMD_Wet_DRUG=df_AMD_Wet_DRUG[df_AMD_Wet_DRUG['Proc_Code']=='J0178']

In [ ]:
#df_AMD_Wet_DRUG.to_csv("../EOC_Raw_Data/drug.csv")

In [ ]:
df_AMD_Wet_Itemized =df_AMD_Wet[df_AMD_Wet['Proc_Clas']=='Non-Drug']

In [ ]:
df_AMD_Wet_Itemized

In [ ]:
df_AMD_Wet_Itemized.to_csv('../EOC_Raw_Data/df_amd_wet_itemized.csv')

In [ ]:
df_AMD_Wet_Itemized_New= (df_AMD_Wet_Itemized.groupby(['Claim_ID','Proc_Clas'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index()
                 )

In [ ]:
df_AMD_Wet_Itemized_New

In [ ]:
Report_AMD_Wet = pd.merge(df_AMD_Wet_DRUG,
                 df_AMD_Wet_Itemized_New,
                 on='Claim_ID', 
                 how='inner')

In [ ]:
d = {'Claim_ID':'Claim_ID','Proc_Code':'Proc_Code','Proc_Desc':'EOC_with_or_no_drug','Sum_Charges_x':'#Units','Sum_Net_Payments_x':'EOC_Total_Pmt_Code_Portion',
         'Sum_Actual_Allowed_Amts_x':'EOC_Total_Calculated_Allow_Code_Portion','Sum_Net_Payments_y':'EOC_Total_Pmt_Non_Code_Portion',
         'Sum_Actual_Allowed_Amts_y':'EOC_Total_Calculated_Allow_Non_Code_Portion','Sum_All_Adj_x':'Total_All_Adj'}

In [ ]:
Report_AMD_Wet = Report_AMD_Wet.rename(columns=lambda col: d.get(str(col)) if str(col) in d else col)

In [ ]:
Report_AMD_Wet[['Claim_ID','Proc_Code','EOC_with_or_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]

In [ ]:
Report_AMD_Wet = Report_AMD_Wet.loc[Report_AMD_Wet['EOC_Total_Pmt_Code_Portion']!=0] 
#result = result.loc[result['Non_Code_Portion_Net_Payments']!=0] 
#result = result.loc[(result['EOC_Total_Pmt_Code_Portion']>= result['Total_All_Adj'])] 

In [ ]:
#Report_AMD_Wet=Report_AMD_Wet[Report_AMD_Wet['Proc_Code']=='J0178']

In [ ]:
#Report_AMD_Wet.to_csv('../EOC_Raw_Data/result.csv',index=False)

In [ ]:
Report_AMD_Wet_New= (Report_AMD_Wet.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)

In [ ]:
Report_AMD_Wet_New

In [ ]:
def eoc_total_pmt(column): 
         return ((column['EOC_Total_Pmt_Code_Portion'] + column['EOC_Total_Pmt_Non_Code_Portion']))


In [ ]:
Report_AMD_Wet_New['EOC_Total_Pmts'] = Report_AMD_Wet_New.apply(eoc_total_pmt, axis=1) 

In [ ]:
def total_code_portion(column): 
    if column['EOC_Total_Pmts'] != 0 :
        return ((column['EOC_Total_Pmt_Code_Portion']/ column['EOC_Total_Pmts'])*100)
    else :
        return 0

In [ ]:
Report_AMD_Wet_New['%_of_code_portion'] = Report_AMD_Wet_New.apply(total_code_portion, axis=1) 

In [ ]:
def total_non_code_portion(column): 
    if column['EOC_Total_Pmts'] != 0 :
        return ((column['EOC_Total_Pmt_Non_Code_Portion']/ column['EOC_Total_Pmts'])*100)
    else :
        return 0

In [ ]:
Report_AMD_Wet_New['%_of_non_code_portion'] = Report_AMD_Wet_New.apply(total_non_code_portion, axis=1) 

In [ ]:
def eoc_total_calc_allow(column): 
         return ((column['EOC_Total_Calculated_Allow_Code_Portion'] + column['EOC_Total_Calculated_Allow_Non_Code_Portion']))

In [ ]:
Report_AMD_Wet_New['EOC_Total_Calc_Allow'] = Report_AMD_Wet_New.apply(eoc_total_calc_allow, axis=1) 

In [ ]:
def total_allow_code_portion(column): 
    if column['EOC_Total_Calc_Allow'] != 0 :
        return ((column['EOC_Total_Calculated_Allow_Code_Portion']/ column['EOC_Total_Calc_Allow'])*100)
    else :
        return 0

In [ ]:
Report_AMD_Wet_New['%_of_allow_code_portion'] = Report_AMD_Wet_New.apply(total_allow_code_portion, axis=1) 

In [ ]:
def total_allow_non_code_portion(column): 
    if column['EOC_Total_Calc_Allow'] != 0 :
        return ((column['EOC_Total_Calculated_Allow_Non_Code_Portion']/ column['EOC_Total_Calc_Allow'])*100)
    else :
        return 0

In [ ]:
Report_AMD_Wet_New['%_of_allow_non_code_portion'] = Report_AMD_Wet_New.apply(total_allow_non_code_portion, axis=1) 

In [ ]:
def eoc_avg_pmt(column): 
    if column['#EOCs'] != 0 :
        return ((column['EOC_Total_Pmts']/ column['#EOCs']))
    else :
        return 0

In [ ]:
Report_AMD_Wet_New['EOC_Avg_Pmt'] = Report_AMD_Wet_New.apply(eoc_avg_pmt, axis=1) 

In [ ]:
def eoc_avg_calc_allow(column): 
    if column['#EOCs'] != 0 :
        return ((column['EOC_Total_Calc_Allow']/ column['#EOCs']))
    else :
        return 0

In [ ]:
Report_AMD_Wet_New['EOC_Avg_Calc_Allow'] = Report_AMD_Wet_New.apply(eoc_avg_calc_allow, axis=1) 

In [ ]:
Report_AMD_Wet_New

In [ ]:
total_amd = Report_AMD_Wet_New.apply(np.sum)
total_amd['Proc_Code'] = 'Total_Drug_EOC'
Report_AMD_Wet_New= Report_AMD_Wet_New.append(pd.DataFrame(total_amd.values, index=total_amd.keys()).T, ignore_index=True)

In [ ]:
Report_AMD_Wet_New

In [ ]:
Report_AMD_Wet_New=Report_AMD_Wet_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

In [ ]:
#csv file name and path
file_name = 'Report_AMD_Wet'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_AMD_Wet_New.to_csv(file_path,index=False)

In [ ]:
#Report_AMD_Wet_Itemized = df_AMD_Wet_Itemized.rename(columns=lambda col: d.get(str(col)) if str(col) in d else col)

In [ ]:
Report_AMD_Wet_Itemized = pd.merge(
                 df_AMD_Wet_Itemized,
                 df_AMD_Wet_DRUG,
                 on='Claim_ID', 
                 how='inner')

In [ ]:
c= {'Claim_ID':'Claim_ID','Proc_Code_x':'Proc_Code','Proc_Desc_x':'Of_EOC_with_drug_Itemized','Sum_Charges_x':'#Units','Sum_Net_Payments_x':'EOC_Total_Pmt_Code_Portion',
         'Sum_Actual_Allowed_Amts_x':'EOC_Total_Calculated_Allow_Code_Portion','Sum_Net_Payments_y':'EOC_Total_Pmt_Non_Code_Portion',
         'Sum_Actual_Allowed_Amts_y':'EOC_Total_Calculated_Allow_Non_Code_Portion','Sum_All_Adj_x':'Total_All_Adj'}

In [ ]:
Report_AMD_Wet_Itemized  = Report_AMD_Wet_Itemized.rename(columns=lambda col: c.get(str(col)) if str(col) in c else col)

In [ ]:
Report_AMD_Wet_Itemized.to_csv("../EOC_Raw_Data/amd_wet_itemized_1.csv")

In [ ]:
Report_AMD_Wet_Itemized

In [ ]:
Report_AMD_Wet_Itemized[['Claim_ID','Proc_Code','Of_EOC_with_drug_Itemized','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]

In [ ]:
Report_AMD_Wet_Itemized.to_csv("../EOC_Raw_Data/amd_wet_itemized_old.csv")

In [ ]:
Report_AMD_Wet_Itemized = Report_AMD_Wet_Itemized.loc[Report_AMD_Wet_Itemized['EOC_Total_Pmt_Code_Portion']!=0]
Report_AMD_Wet_Itemized = Report_AMD_Wet_Itemized.loc[Report_AMD_Wet_Itemized['EOC_Total_Pmt_Non_Code_Portion']!=0]

In [ ]:
Report_AMD_Wet_Itemized.to_csv("../EOC_Raw_Data/amd_wet_itemized.csv")

In [ ]:
Report_AMD_Wet_Itemized_New= (Report_AMD_Wet_Itemized.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)

In [ ]:
Report_AMD_Wet_Itemized_New

In [ ]:
Report_AMD_Wet_Itemized_New['EOC_Total_Pmts'] = Report_AMD_Wet_Itemized_New.apply(eoc_total_pmt, axis=1) 

In [ ]:
Report_AMD_Wet_Itemized_New['%_of_code_portion'] = Report_AMD_Wet_Itemized_New.apply(total_code_portion, axis=1) 

In [ ]:
Report_AMD_Wet_Itemized_New['%_of_non_code_portion'] = Report_AMD_Wet_Itemized_New.apply(total_non_code_portion, axis=1) 

In [ ]:
Report_AMD_Wet_Itemized_New['EOC_Total_Calc_Allow'] = Report_AMD_Wet_Itemized_New.apply(eoc_total_calc_allow, axis=1) 

In [ ]:
Report_AMD_Wet_Itemized_New['%_of_allow_code_portion'] = Report_AMD_Wet_Itemized_New.apply(total_allow_code_portion, axis=1) 

In [ ]:
Report_AMD_Wet_Itemized_New['%_of_allow_non_code_portion'] = Report_AMD_Wet_Itemized_New.apply(total_allow_non_code_portion, axis=1) 

In [ ]:
Report_AMD_Wet_Itemized_New['EOC_Avg_Pmt'] = Report_AMD_Wet_Itemized_New.apply(eoc_avg_pmt, axis=1) 

In [ ]:
Report_AMD_Wet_Itemized_New['EOC_Avg_Calc_Allow'] = Report_AMD_Wet_Itemized_New.apply(eoc_avg_calc_allow, axis=1) 

In [ ]:
total_iamd = Report_AMD_Wet_Itemized_New.apply(np.sum)
total_iamd['Proc_Code'] = 'Total_Drug_Itemized_EOC'
Report_AMD_Wet_Itemized_New= Report_AMD_Wet_Itemized_New.append(pd.DataFrame(total_iamd.values, index=total_iamd.keys()).T, ignore_index=True)

In [ ]:
Report_AMD_Wet_Itemized_New=Report_AMD_Wet_Itemized_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

In [ ]:
#csv file name and path
file_name = 'Report_Itemized_AMD_Wet'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_AMD_Wet_Itemized_New.to_csv(file_path,index=False)


################################################################################################################################
# Lumped Non Drug Procedures
################################################################################################################################


In [ ]:
df_Lumped_NonDrug.info()

In [ ]:
df_Lumped_AMD_Wet=df_Lumped_NonDrug.loc[(df_Lumped_NonDrug['Primary_Diagnosis_10'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code02'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code03'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code04'] == 'AMD - Wet')
                                        | (df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code05'] == 'AMD - Wet') 
                                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code06'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code07'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code08'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code09'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code010'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code011'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['Primary_Diagnosis_9'] == 'AMD - Wet') 
                                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code02'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code03'] == 'AMD - Wet')
                                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code04'] == 'AMD - Wet')]

In [ ]:
b= {'Claim_ID':'#EOCs','Proc_Code':'Proc_Code','Proc_Desc':'EOC_with_no_drug','Sum_Charges':'#Units','Sum_Net_Payments':'EOC_Total_Pmt_Code_Portion',
         'Sum_Actual_Allowed_Amts':'EOC_Total_Calculated_Allow_Code_Portion'}

In [ ]:
Report_Lumped_AMD_Wet  = df_Lumped_AMD_Wet .rename(columns=lambda col: b.get(str(col)) if str(col) in b else col)

In [ ]:
Report_Lumped_AMD_Wet[['#EOCs','EOC_with_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion',
         ]]

In [ ]:
Report_Lumped_AMD_Wet= Report_Lumped_AMD_Wet.loc[Report_Lumped_AMD_Wet['EOC_Total_Pmt_Code_Portion']!=0]

In [ ]:
Report_Lumped_AMD_Wet['#EOCs'].nunique()

In [ ]:
Report_Lumped_AMD_Wet.agg(['sum'])

################################################################################################################################
# CRVO
################################################################################################################################

In [ ]:
df_CRVO=df_Modifier.loc[((df_Modifier['Primary_Diagnosis_10'] == 'CRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code02'] == 'CRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code03'] == 'CRVO')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code02'] == 'CRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code03'] == 'CRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code04'] == 'CRVO')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code05'] == 'CRVO') |(df_Modifier['ICD-10_Claim_Diagnosis_Code06'] == 'CRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code07'] == 'CRVO')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code08'] == 'CRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code09'] == 'CRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code010'] == 'CRVO')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code011'] == 'CRVO')|(df_Modifier['Primary_Diagnosis_9'] == 'CRVO')
                        |(df_Modifier['ICD-9_Claim_Diagnosis_Code02'] == 'CRVO')
                        |(df_Modifier['ICD-9_Claim_Diagnosis_Code03'] == 'CRVO')|(df_Modifier['ICD-9_Claim_Diagnosis_Code04'] == 'CRVO'))
                        &((df_Modifier['ICD-10_Claim_Diagnosis_Code02'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code03'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code04'] == 'ME')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code05'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code06'] == 'ME')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code07'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code08'] == 'ME')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code09'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code010'] == 'ME')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code011'] == 'ME')
                        |(df_Modifier['Primary_Diagnosis_9'] == 'ME')|(df_Modifier['ICD-9_Claim_Diagnosis_Code02'] == 'ME')
                        |(df_Modifier['ICD-9_Claim_Diagnosis_Code03'] == 'ME') |(df_Modifier['ICD-9_Claim_Diagnosis_Code04'] == 'ME'))]
         

In [ ]:
df_CRVO=(df_CRVO.groupby(['Claim_ID','Proc_Clas','Proc_Code','Proc_Desc'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index())
df_CRVO


In [ ]:
df_CRVO_DRUG=df_CRVO[df_CRVO['Proc_Clas']=='Drug']
df_CRVO_DRUG

In [ ]:
df_CRVO_Itemized =df_CRVO[df_CRVO['Proc_Clas']=='Non-Drug']
df_CRVO_Itemized

In [ ]:
df_CRVO_Itemized_New= (df_CRVO_Itemized.groupby(['Claim_ID','Proc_Clas'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index()
                 )


In [ ]:
Report_CRVO = pd.merge(df_CRVO_DRUG,
                 df_CRVO_Itemized_New,
                 on='Claim_ID', 
                 how='inner')

In [ ]:
Report_CRVO

In [ ]:
Report_CRVO = Report_CRVO.rename(columns=lambda col: d.get(str(col)) if str(col) in d else col)
Report_CRVO[['Claim_ID','Proc_Code','EOC_with_or_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]

In [ ]:
Report_CRVO

In [ ]:
Report_CRVO = Report_CRVO.loc[Report_CRVO['EOC_Total_Pmt_Code_Portion']!=0] 

In [ ]:
Report_CRVO

In [ ]:
Report_CRVO_New= (Report_CRVO.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)


In [ ]:
Report_CRVO_New['EOC_Total_Pmts'] = Report_CRVO_New.apply(eoc_total_pmt, axis=1) 
Report_CRVO_New['%_of_code_portion'] = Report_CRVO_New.apply(total_code_portion, axis=1) 
Report_CRVO_New['%_of_non_code_portion'] = Report_CRVO_New.apply(total_non_code_portion, axis=1) 
Report_CRVO_New['EOC_Total_Calc_Allow'] = Report_CRVO_New.apply(eoc_total_calc_allow, axis=1) 
Report_CRVO_New['%_of_allow_code_portion'] = Report_CRVO_New.apply(total_allow_code_portion, axis=1) 
Report_CRVO_New['%_of_allow_non_code_portion'] = Report_CRVO_New.apply(total_allow_non_code_portion, axis=1) 
Report_CRVO_New['EOC_Avg_Pmt'] = Report_CRVO_New.apply(eoc_avg_pmt, axis=1) 
Report_CRVO_New['EOC_Avg_Calc_Allow'] = Report_CRVO_New.apply(eoc_avg_calc_allow, axis=1) 


In [ ]:
total_crvo = Report_CRVO_New.apply(np.sum)
total_crvo['Proc_Code'] = 'Total_Drug_EOC'
Report_CRVO_New= Report_CRVO_New.append(pd.DataFrame(total_crvo.values, index=total_crvo.keys()).T, ignore_index=True,sort=True)

In [ ]:
Report_CRVO_New=Report_CRVO_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

In [ ]:
Report_CRVO_New

In [ ]:
#csv file name and path
file_name = 'Report_CRVO'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_CRVO_New.to_csv(file_path,index=False)

In [ ]:
Report_CRVO_Itemized = pd.merge(
                 df_CRVO_Itemized,
                 df_CRVO_DRUG,
                 on='Claim_ID', 
                 how='inner')

In [ ]:
Report_CRVO_Itemized  = Report_CRVO_Itemized .rename(columns=lambda col: c.get(str(col)) if str(col) in c else col)

In [ ]:
Report_CRVO_Itemized[['Claim_ID','Of_EOC_with_drug_Itemized','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]

In [ ]:
Report_CRVO_Itemized = Report_CRVO_Itemized.loc[Report_CRVO_Itemized['EOC_Total_Pmt_Code_Portion']!=0]

In [ ]:
Report_CRVO_Itemized_New= (Report_CRVO_Itemized.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)

In [ ]:
Report_CRVO_Itemized_New['EOC_Total_Pmts'] = Report_CRVO_Itemized_New.apply(eoc_total_pmt, axis=1) 
Report_CRVO_Itemized_New['%_of_code_portion'] = Report_CRVO_Itemized_New.apply(total_code_portion, axis=1) 
Report_CRVO_Itemized_New['%_of_non_code_portion'] = Report_CRVO_Itemized_New.apply(total_non_code_portion, axis=1) 
Report_CRVO_Itemized_New['EOC_Total_Calc_Allow'] = Report_CRVO_Itemized_New.apply(eoc_total_calc_allow, axis=1) 
Report_CRVO_Itemized_New['%_of_allow_code_portion'] = Report_CRVO_Itemized_New.apply(total_allow_code_portion, axis=1) 
Report_CRVO_Itemized_New['%_of_allow_non_code_portion'] = Report_CRVO_Itemized_New.apply(total_allow_non_code_portion, axis=1) 
Report_CRVO_Itemized_New['EOC_Avg_Pmt'] = Report_CRVO_Itemized_New.apply(eoc_avg_pmt, axis=1) 
Report_CRVO_Itemized_New['EOC_Avg_Calc_Allow'] = Report_CRVO_Itemized_New.apply(eoc_avg_calc_allow, axis=1) 

In [ ]:
total_icrvo = Report_CRVO_Itemized_New.apply(np.sum)
total_icrvo['Proc_Code'] = 'Total_Drug_Itemized_EOC'
Report_CRVO_Itemized_New= Report_CRVO_Itemized_New.append(pd.DataFrame(total_icrvo.values, index=total_icrvo.keys()).T, ignore_index=True,sort=True)

In [ ]:
Report_CRVO_Itemized_New=Report_CRVO_Itemized_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

In [ ]:
Report_CRVO_Itemized_New

In [ ]:
#csv file name and path
file_name = 'Report_Itemized_CRVO'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_CRVO_Itemized_New.to_csv(file_path,index=False)

In [ ]:
df_Lumped_CRVO=df_Lumped_NonDrug.loc[((df_Lumped_NonDrug['Primary_Diagnosis_10'] == 'CRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code02'] == 'CRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code03'] == 'CRVO')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code02'] == 'CRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code03'] == 'CRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code04'] == 'CRVO')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code05'] == 'CRVO') |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code06'] == 'CRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code07'] == 'CRVO')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code08'] == 'CRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code09'] == 'CRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code010'] == 'CRVO')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code011'] == 'CRVO')|(df_Lumped_NonDrug['Primary_Diagnosis_9'] == 'CRVO')
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code02'] == 'CRVO')
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code03'] == 'CRVO') 
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code04'] == 'CRVO'))
                        &((df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code02'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code03'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code04'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code05'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code06'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code07'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code08'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code09'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code010'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code011'] == 'ME')
                        |(df_Lumped_NonDrug['Primary_Diagnosis_9'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code02'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code03'] == 'ME') 
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code04'] == 'ME'))]
         

In [ ]:
Report_Lumped_CRVO  = df_Lumped_CRVO.rename(columns=lambda col: b.get(str(col)) if str(col) in b else col)

In [ ]:
Report_Lumped_CRVO[['#EOCs','EOC_with_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion',
         ]]

In [ ]:
Report_Lumped_CRVO= Report_Lumped_CRVO.loc[Report_Lumped_CRVO['EOC_Total_Pmt_Code_Portion']!=0]

In [ ]:
Report_Lumped_CRVO['#EOCs'].nunique()

In [ ]:
Report_Lumped_CRVO.agg(['sum'])

################################################################################################################################
# BRVO
################################################################################################################################

In [ ]:
df_BRVO=df_Modifier.loc[((df_Modifier['Primary_Diagnosis_10'] == 'BRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code02'] == 'BRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code03'] == 'BRVO')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code02'] == 'BRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code03'] == 'BRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code04'] == 'BRVO')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code05'] == 'BRVO') |(df_Modifier['ICD-10_Claim_Diagnosis_Code06'] == 'BRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code07'] == 'BRVO')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code08'] == 'BRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code09'] == 'BRVO')|(df_Modifier['ICD-10_Claim_Diagnosis_Code010'] == 'BRVO')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code011'] == 'BRVO')|(df_Modifier['Primary_Diagnosis_9'] == 'CRVO')
                        |(df_Modifier['ICD-9_Claim_Diagnosis_Code02'] == 'BRVO')
                        |(df_Modifier['ICD-9_Claim_Diagnosis_Code03'] == 'BRVO') 
                        |(df_Modifier['ICD-9_Claim_Diagnosis_Code04'] == 'BRVO'))
                        &((df_Modifier['ICD-10_Claim_Diagnosis_Code02'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code03'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code04'] == 'ME')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code05'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code06'] == 'ME')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code07'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code08'] == 'ME')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code09'] == 'ME')|(df_Modifier['ICD-10_Claim_Diagnosis_Code010'] == 'ME')
                        |(df_Modifier['ICD-10_Claim_Diagnosis_Code011'] == 'ME')
                        |(df_Modifier['Primary_Diagnosis_9'] == 'ME')
                        |(df_Modifier['ICD-9_Claim_Diagnosis_Code02'] == 'ME')
                        |(df_Modifier['ICD-9_Claim_Diagnosis_Code03'] == 'ME') 
                        |(df_Modifier['ICD-9_Claim_Diagnosis_Code04'] == 'ME'))]
         

In [ ]:
df_BRVO=(df_BRVO.groupby(['Claim_ID','Proc_Clas','Proc_Code','Proc_Desc'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index())
df_BRVO


In [ ]:
df_BRVO_DRUG=df_BRVO[df_BRVO['Proc_Clas']=='Drug']
df_BRVO_DRUG

In [ ]:
df_BRVO_Itemized =df_BRVO[df_BRVO['Proc_Clas']=='Non-Drug']
df_BRVO_Itemized

In [ ]:
df_BRVO_Itemized_New= (df_BRVO_Itemized.groupby(['Claim_ID','Proc_Clas'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index()
                 )


In [ ]:
Report_BRVO = pd.merge(df_BRVO_DRUG,
                 df_BRVO_Itemized_New,
                 on='Claim_ID', 
                 how='inner')

In [ ]:
Report_BRVO = Report_BRVO.rename(columns=lambda col: d.get(str(col)) if str(col) in d else col)
Report_BRVO[['Claim_ID','Proc_Code','EOC_with_or_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]



In [ ]:
Report_BRVO = Report_BRVO.loc[Report_BRVO['EOC_Total_Pmt_Code_Portion']!=0] 

In [ ]:
Report_BRVO_New= (Report_BRVO.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)

In [ ]:
Report_BRVO_New['EOC_Total_Pmts'] = Report_BRVO_New.apply(eoc_total_pmt, axis=1) 
Report_BRVO_New['%_of_code_portion'] = Report_BRVO_New.apply(total_code_portion, axis=1) 
Report_BRVO_New['%_of_non_code_portion'] = Report_BRVO_New.apply(total_non_code_portion, axis=1) 
Report_BRVO_New['EOC_Total_Calc_Allow'] = Report_BRVO_New.apply(eoc_total_calc_allow, axis=1) 
Report_BRVO_New['%_of_allow_code_portion'] = Report_BRVO_New.apply(total_allow_code_portion, axis=1) 
Report_BRVO_New['%_of_allow_non_code_portion'] = Report_BRVO_New.apply(total_allow_non_code_portion, axis=1) 
Report_BRVO_New['EOC_Avg_Pmt'] = Report_BRVO_New.apply(eoc_avg_pmt, axis=1) 
Report_BRVO_New['EOC_Avg_Calc_Allow'] = Report_BRVO_New.apply(eoc_avg_calc_allow, axis=1) 


In [ ]:
total_brvo = Report_BRVO_New.apply(np.sum)
total_brvo['Proc_Code'] = 'Total_Drug_EOC'
Report_BRVO_New= Report_BRVO_New.append(pd.DataFrame(total_brvo.values, index=total_brvo.keys()).T, ignore_index=True,sort=True)

In [ ]:
Report_BRVO_New=Report_BRVO_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

In [ ]:
Report_BRVO_New

In [ ]:
#csv file name and path
file_name = 'Report_BRVO'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_BRVO_New.to_csv(file_path,index=False)

In [ ]:
Report_BRVO_Itemized = pd.merge(
                 df_BRVO_Itemized,
                 df_BRVO_DRUG,
                 on='Claim_ID', 
                 how='inner')

In [ ]:
Report_BRVO_Itemized  = Report_BRVO_Itemized .rename(columns=lambda col: c.get(str(col)) if str(col) in c else col)

In [ ]:
Report_BRVO_Itemized[['Claim_ID','Of_EOC_with_drug_Itemized','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]

In [ ]:
Report_BRVO_Itemized = Report_BRVO_Itemized.loc[Report_BRVO_Itemized['EOC_Total_Pmt_Code_Portion']!=0]

In [ ]:
Report_BRVO_Itemized_New= (Report_BRVO_Itemized.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)

In [ ]:
Report_BRVO_Itemized_New['EOC_Total_Pmts'] = Report_BRVO_Itemized_New.apply(eoc_total_pmt, axis=1) 
Report_BRVO_Itemized_New['%_of_code_portion'] = Report_BRVO_Itemized_New.apply(total_code_portion, axis=1) 
Report_BRVO_Itemized_New['%_of_non_code_portion'] = Report_BRVO_Itemized_New.apply(total_non_code_portion, axis=1) 
Report_BRVO_Itemized_New['EOC_Total_Calc_Allow'] = Report_BRVO_Itemized_New.apply(eoc_total_calc_allow, axis=1) 
Report_BRVO_Itemized_New['%_of_allow_code_portion'] = Report_BRVO_Itemized_New.apply(total_allow_code_portion, axis=1) 
Report_BRVO_Itemized_New['%_of_allow_non_code_portion'] = Report_BRVO_Itemized_New.apply(total_allow_non_code_portion, axis=1) 
Report_BRVO_Itemized_New['EOC_Avg_Pmt'] = Report_BRVO_Itemized_New.apply(eoc_avg_pmt, axis=1) 
Report_BRVO_Itemized_New['EOC_Avg_Calc_Allow'] = Report_BRVO_Itemized_New.apply(eoc_avg_calc_allow, axis=1) 

In [ ]:
total_ibrvo = Report_BRVO_Itemized_New.apply(np.sum)
total_ibrvo['Proc_Code'] = 'Total_Drug_Itemized_EOC'
Report_BRVO_Itemized_New= Report_BRVO_Itemized_New.append(pd.DataFrame(total_ibrvo.values, index=total_ibrvo.keys()).T, ignore_index=True,sort=True)

In [ ]:
Report_BRVO_Itemized_New=Report_BRVO_Itemized_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

In [ ]:
Report_BRVO_Itemized_New

In [ ]:
#csv file name and path
file_name = 'Report_Itemized_BRVO'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_BRVO_Itemized_New.to_csv(file_path,index=False)

In [ ]:
df_Lumped_BRVO=df_Lumped_NonDrug.loc[((df_Lumped_NonDrug['Primary_Diagnosis_10'] == 'BRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code02'] == 'BRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code03'] == 'BRVO')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code02'] == 'BRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code03'] == 'BRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code04'] == 'BRVO')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code05'] == 'BRVO') |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code06'] == 'BRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code07'] == 'BRVO')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code08'] == 'BRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code09'] == 'BRVO')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code010'] == 'BRVO')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code011'] == 'BRVO')|(df_Lumped_NonDrug['Primary_Diagnosis_9'] == 'BRVO')
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code02'] == 'BRVO')
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code03'] == 'BRVO') 
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code04'] == 'BRVO'))
                        &((df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code02'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code03'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code04'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code05'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code06'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code07'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code08'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code09'] == 'ME')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code010'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code011'] == 'ME')
                        |(df_Lumped_NonDrug['Primary_Diagnosis_9'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code02'] == 'ME')
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code03'] == 'ME') 
                        |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code04'] == 'ME'))]
         

In [ ]:
Report_Lumped_BRVO  = df_Lumped_BRVO.rename(columns=lambda col: b.get(str(col)) if str(col) in b else col)

In [ ]:
Report_Lumped_BRVO[['#EOCs','EOC_with_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion',
         ]]

In [ ]:
Report_Lumped_BRVO= Report_Lumped_BRVO.loc[Report_Lumped_BRVO['EOC_Total_Pmt_Code_Portion']!=0]

In [ ]:
Report_Lumped_BRVO['#EOCs'].nunique()

In [ ]:
Report_Lumped_BRVO.agg(['sum'])

################################################################################################################################
# Diabetes
################################################################################################################################

In [ ]:
df_Diabetes=df_Modifier.loc[(df_Modifier['Primary_Diagnosis_10'] == 'Diabetes') |(df_Modifier['ICD-10_Claim_Diagnosis_Code02'] == 'Diabetes')
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code03'] == 'Diabetes')|(df_Modifier['ICD-10_Claim_Diagnosis_Code04'] == 'Diabetes') 
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code05'] == 'Diabetes') |(df_Modifier['ICD-10_Claim_Diagnosis_Code06'] == 'Diabetes')
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code07'] == 'Diabetes')|(df_Modifier['ICD-10_Claim_Diagnosis_Code08'] == 'Diabetes')
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code09'] == 'Diabetes')|(df_Modifier['ICD-10_Claim_Diagnosis_Code010'] == 'Diabetes')
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code011'] == 'Diabetes')|(df_Modifier['Primary_Diagnosis_9'] == 'Diabetes')
                           |(df_Modifier['ICD-9_Claim_Diagnosis_Code02'] == 'Diabetes')
                           |(df_Modifier['ICD-9_Claim_Diagnosis_Code03'] == 'Diabetes') 
                           |(df_Modifier['ICD-9_Claim_Diagnosis_Code04'] == 'Diabetes')]

In [ ]:
df_Diabetes=(df_Diabetes.groupby(['Claim_ID','Proc_Clas','Proc_Code','Proc_Desc'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index())
df_Diabetes


In [ ]:
df_Diabetes_DRUG=df_Diabetes[df_Diabetes['Proc_Clas']=='Drug']
df_Diabetes_DRUG

In [ ]:
df_Diabetes_Itemized =df_Diabetes[df_Diabetes['Proc_Clas']=='Non-Drug']
df_Diabetes_Itemized

In [ ]:
df_Diabetes_Itemized_New= (df_Diabetes_Itemized.groupby(['Claim_ID','Proc_Clas'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index()
                 )


In [ ]:
Report_Diabetes = pd.merge(df_Diabetes_DRUG,
                 df_Diabetes_Itemized_New,
                 on='Claim_ID', 
                 how='inner')

In [ ]:
Report_Diabetes = Report_Diabetes.rename(columns=lambda col: d.get(str(col)) if str(col) in d else col)
Report_Diabetes[['Claim_ID','Proc_Code','EOC_with_or_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]



In [ ]:
Report_Diabetes = Report_Diabetes.loc[Report_Diabetes['EOC_Total_Pmt_Code_Portion']!=0] 

In [ ]:
Report_Diabetes_New= (Report_Diabetes.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)

In [ ]:
Report_Diabetes_New['EOC_Total_Pmts'] = Report_Diabetes_New.apply(eoc_total_pmt, axis=1) 
Report_Diabetes_New['%_of_code_portion'] = Report_Diabetes_New.apply(total_code_portion, axis=1) 
Report_Diabetes_New['%_of_non_code_portion'] = Report_Diabetes_New.apply(total_non_code_portion, axis=1) 
Report_Diabetes_New['EOC_Total_Calc_Allow'] = Report_Diabetes_New.apply(eoc_total_calc_allow, axis=1) 
Report_Diabetes_New['%_of_allow_code_portion'] = Report_Diabetes_New.apply(total_allow_code_portion, axis=1) 
Report_Diabetes_New['%_of_allow_non_code_portion'] = Report_Diabetes_New.apply(total_allow_non_code_portion, axis=1) 
Report_Diabetes_New['EOC_Avg_Pmt'] = Report_Diabetes_New.apply(eoc_avg_pmt, axis=1) 
Report_Diabetes_New['EOC_Avg_Calc_Allow'] = Report_Diabetes_New.apply(eoc_avg_calc_allow, axis=1) 


In [ ]:
total_diabetes = Report_Diabetes_New.apply(np.sum)
total_diabetes['Proc_Code'] = 'Total_Drug_EOC'
Report_Diabetes_New= Report_Diabetes_New.append(pd.DataFrame(total_diabetes.values, index=total_diabetes.keys()).T, ignore_index=True,sort=True)

In [ ]:
Report_Diabetes_New=Report_Diabetes_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

In [ ]:
Report_Diabetes_New

In [ ]:
#csv file name and path
file_name = 'Report_Diabetes'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_Diabetes_New.to_csv(file_path,index=False)

In [ ]:
Report_Diabetes_Itemized = pd.merge(
                 df_Diabetes_Itemized,
                 df_Diabetes_DRUG,
                 on='Claim_ID', 
                 how='inner')

In [ ]:
Report_Diabetes_Itemized  = Report_Diabetes_Itemized .rename(columns=lambda col: c.get(str(col)) if str(col) in c else col)

In [ ]:
Report_Diabetes_Itemized[['Claim_ID','Of_EOC_with_drug_Itemized','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]

In [ ]:
Report_Diabetes_Itemized = Report_Diabetes_Itemized.loc[Report_Diabetes_Itemized['EOC_Total_Pmt_Code_Portion']!=0]

In [ ]:
Report_Diabetes_Itemized_New= (Report_Diabetes_Itemized.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)

In [ ]:
Report_Diabetes_Itemized_New['EOC_Total_Pmts'] = Report_Diabetes_Itemized_New.apply(eoc_total_pmt, axis=1) 
Report_Diabetes_Itemized_New['%_of_code_portion'] = Report_Diabetes_Itemized_New.apply(total_code_portion, axis=1) 
Report_Diabetes_Itemized_New['%_of_non_code_portion'] = Report_Diabetes_Itemized_New.apply(total_non_code_portion, axis=1) 
Report_Diabetes_Itemized_New['EOC_Total_Calc_Allow'] = Report_Diabetes_Itemized_New.apply(eoc_total_calc_allow, axis=1) 
Report_Diabetes_Itemized_New['%_of_allow_code_portion'] = Report_Diabetes_Itemized_New.apply(total_allow_code_portion, axis=1) 
Report_Diabetes_Itemized_New['%_of_allow_non_code_portion'] = Report_Diabetes_Itemized_New.apply(total_allow_non_code_portion, axis=1) 
Report_Diabetes_Itemized_New['EOC_Avg_Pmt'] = Report_Diabetes_Itemized_New.apply(eoc_avg_pmt, axis=1) 
Report_Diabetes_Itemized_New['EOC_Avg_Calc_Allow'] = Report_Diabetes_Itemized_New.apply(eoc_avg_calc_allow, axis=1) 

In [ ]:
total_idiabetes = Report_Diabetes_Itemized_New.apply(np.sum)
total_idiabetes['Proc_Code'] = 'Total_Drug_Itemized_EOC'
Report_Diabetes_Itemized_New= Report_Diabetes_Itemized_New.append(pd.DataFrame(total_idiabetes.values, index=total_idiabetes.keys()).T, ignore_index=True,sort=True)

In [ ]:
Report_Diabetes_Itemized_New=Report_Diabetes_Itemized_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

In [ ]:
Report_Diabetes_Itemized_New

In [ ]:
#csv file name and path
file_name = 'Report_Itemized_Diabetes'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_Diabetes_Itemized_New.to_csv(file_path,index=False)

In [ ]:
df_Lumped_Diabetes=df_Lumped_NonDrug.loc[(df_Lumped_NonDrug['Primary_Diagnosis_10'] == 'Diabetes') |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code02'] == 'Diabetes')
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code03'] == 'Diabetes')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code04'] == 'Diabetes') 
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code05'] == 'Diabetes') |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code06'] == 'Diabetes')
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code07'] == 'Diabetes')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code08'] == 'Diabetes')
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code09'] == 'Diabetes')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code010'] == 'Diabetes')
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code011'] == 'Diabetes')|(df_Lumped_NonDrug['Primary_Diagnosis_9'] == 'Diabetes')
                           |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code02'] == 'Diabetes')
                           |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code03'] == 'Diabetes') 
                           |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code04'] == 'Diabetes')]

In [ ]:
Report_Lumped_Diabetes  = df_Lumped_Diabetes.rename(columns=lambda col: b.get(str(col)) if str(col) in b else col)

In [ ]:
Report_Lumped_Diabetes[['#EOCs','EOC_with_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion',
         ]]

In [ ]:
Report_Lumped_Diabetes= Report_Lumped_Diabetes.loc[Report_Lumped_Diabetes['EOC_Total_Pmt_Code_Portion']!=0]

In [ ]:
Report_Lumped_Diabetes['#EOCs'].nunique()

In [ ]:
Report_Lumped_Diabetes.agg(['sum'])

################################################################################################################################
# Other
################################################################################################################################

In [ ]:
df_Other=df_Modifier.loc[(df_Modifier['Primary_Diagnosis_10'] == 'OTHER') |(df_Modifier['ICD-10_Claim_Diagnosis_Code02'] == 'OTHER')
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code03'] == 'OTHER')|(df_Modifier['ICD-10_Claim_Diagnosis_Code04'] == 'OTHER') 
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code05'] == 'OTHER') |(df_Modifier['ICD-10_Claim_Diagnosis_Code06'] == 'OTHER')
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code07'] == 'OTHER')|(df_Modifier['ICD-10_Claim_Diagnosis_Code08'] == 'OTHER')
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code09'] == 'OTHER')|(df_Modifier['ICD-10_Claim_Diagnosis_Code010'] == 'OTHER')
                           |(df_Modifier['ICD-10_Claim_Diagnosis_Code011'] == 'OTHER')|(df_Modifier['Primary_Diagnosis_9'] == 'OTHER')
                           |(df_Modifier['ICD-9_Claim_Diagnosis_Code02'] == 'OTHER')
                           |(df_Modifier['ICD-9_Claim_Diagnosis_Code03'] == 'OTHER') 
                           |(df_Modifier['ICD-9_Claim_Diagnosis_Code04'] == 'OTHER')]

df_Other=(df_Other.groupby(['Claim_ID','Proc_Clas','Proc_Code','Proc_Desc'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index())
df_Other


df_Other_DRUG=df_Other[df_Other['Proc_Clas']=='Drug']
df_Other_DRUG

df_Other_Itemized =df_Other[df_Other['Proc_Clas']=='Non-Drug']
df_Other_Itemized

df_Other_Itemized_New= (df_Other_Itemized.groupby(['Claim_ID','Proc_Clas'])
    .agg({'Sum_Charges':'sum','Sum_All_Charges':'sum','Sum_Net_Payments':'sum',
         'Sum_Contractual_Adj':'sum','Sum_Calculated_Allowable':'sum','Sum_Actual_Allowed_Amts':'sum','Sum_Expected_Allowed_Amts':'sum','Sum_All_Adj':'sum','Sum_Allowable_Diff':'sum','Sum_Payments':'sum',
          })
    .reset_index()
                 )


Report_Other = pd.merge(df_Other_DRUG,
                 df_Other_Itemized_New,
                 on='Claim_ID', 
                 how='inner')

Report_Other = Report_Other.rename(columns=lambda col: d.get(str(col)) if str(col) in d else col)
Report_Other[['Claim_ID','Proc_Code','EOC_with_or_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]



Report_Other = Report_Other.loc[Report_Other['EOC_Total_Pmt_Code_Portion']!=0] 

Report_Other_New= (Report_Other.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)

Report_Other_New['EOC_Total_Pmts'] = Report_Other_New.apply(eoc_total_pmt, axis=1) 
Report_Other_New['%_of_code_portion'] = Report_Other_New.apply(total_code_portion, axis=1) 
Report_Other_New['%_of_non_code_portion'] = Report_Other_New.apply(total_non_code_portion, axis=1) 
Report_Other_New['EOC_Total_Calc_Allow'] = Report_Other_New.apply(eoc_total_calc_allow, axis=1) 
Report_Other_New['%_of_allow_code_portion'] = Report_Other_New.apply(total_allow_code_portion, axis=1) 
Report_Other_New['%_of_allow_non_code_portion'] = Report_Other_New.apply(total_allow_non_code_portion, axis=1) 
Report_Other_New['EOC_Avg_Pmt'] = Report_Other_New.apply(eoc_avg_pmt, axis=1) 
Report_Other_New['EOC_Avg_Calc_Allow'] = Report_Other_New.apply(eoc_avg_calc_allow, axis=1) 


total_Other = Report_Other_New.apply(np.sum)
total_Other['Proc_Code'] = 'Total_Drug_EOC'
Report_Other_New= Report_Other_New.append(pd.DataFrame(total_Other.values, index=total_Other.keys()).T, ignore_index=True,sort=True)

Report_Other_New=Report_Other_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

Report_Other_New

#csv file name and path
file_name = 'Report_Other'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_Other_New.to_csv(file_path,index=False)

Report_Other_Itemized = pd.merge(
                 df_Other_Itemized,
                 df_Other_DRUG,
                 on='Claim_ID', 
                 how='inner')

Report_Other_Itemized  = Report_Other_Itemized .rename(columns=lambda col: c.get(str(col)) if str(col) in c else col)

Report_Other_Itemized[['Claim_ID','Of_EOC_with_drug_Itemized','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
         'EOC_Total_Calculated_Allow_Non_Code_Portion','Total_All_Adj']]

Report_Other_Itemized = Report_Other_Itemized.loc[Report_Other_Itemized['EOC_Total_Pmt_Code_Portion']!=0]

Report_Other_Itemized_New= (Report_Other_Itemized.groupby(['Proc_Code'])
    .agg({'Claim_ID':'count','#Units':'sum','EOC_Total_Pmt_Code_Portion':'sum','EOC_Total_Pmt_Non_Code_Portion':'sum',
         'EOC_Total_Calculated_Allow_Code_Portion':'sum','EOC_Total_Calculated_Allow_Non_Code_Portion':'sum',
         'Total_All_Adj':'sum'})
    .reset_index()
    .rename(columns={'Claim_ID':'#EOCs'})
)

Report_Other_Itemized_New['EOC_Total_Pmts'] = Report_Other_Itemized_New.apply(eoc_total_pmt, axis=1) 
Report_Other_Itemized_New['%_of_code_portion'] = Report_Other_Itemized_New.apply(total_code_portion, axis=1) 
Report_Other_Itemized_New['%_of_non_code_portion'] = Report_Other_Itemized_New.apply(total_non_code_portion, axis=1) 
Report_Other_Itemized_New['EOC_Total_Calc_Allow'] = Report_Other_Itemized_New.apply(eoc_total_calc_allow, axis=1) 
Report_Other_Itemized_New['%_of_allow_code_portion'] = Report_Other_Itemized_New.apply(total_allow_code_portion, axis=1) 
Report_Other_Itemized_New['%_of_allow_non_code_portion'] = Report_Other_Itemized_New.apply(total_allow_non_code_portion, axis=1) 
Report_Other_Itemized_New['EOC_Avg_Pmt'] = Report_Other_Itemized_New.apply(eoc_avg_pmt, axis=1) 
Report_Other_Itemized_New['EOC_Avg_Calc_Allow'] = Report_Other_Itemized_New.apply(eoc_avg_calc_allow, axis=1) 

total_iOther = Report_Other_Itemized_New.apply(np.sum)
total_iOther['Proc_Code'] = 'Total_Drug_Itemized_EOC'
Report_Other_Itemized_New= Report_Other_Itemized_New.append(pd.DataFrame(total_iOther.values, index=total_iOther.keys()).T, ignore_index=True,sort=True)

Report_Other_Itemized_New=Report_Other_Itemized_New[['Proc_Code','#EOCs','#Units','EOC_Total_Pmts','EOC_Total_Pmt_Code_Portion','EOC_Total_Pmt_Non_Code_Portion',
                             '%_of_code_portion','%_of_non_code_portion','EOC_Avg_Pmt','EOC_Total_Calc_Allow','EOC_Total_Calculated_Allow_Code_Portion',
                             'EOC_Total_Calculated_Allow_Non_Code_Portion','%_of_allow_code_portion','%_of_allow_non_code_portion','EOC_Avg_Calc_Allow'
                            ]]

Report_Other_Itemized_New

#csv file name and path
file_name = 'Report_Itemized_Other'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
Report_Other_Itemized_New.to_csv(file_path,index=False)

df_Lumped_Other=df_Lumped_NonDrug.loc[(df_Lumped_NonDrug['Primary_Diagnosis_10'] == 'OTHER') |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code02'] == 'OTHER')
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code03'] == 'OTHER')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code04'] == 'OTHER') 
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code05'] == 'OTHER') |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code06'] == 'OTHER')
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code07'] == 'OTHER')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code08'] == 'OTHER')
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code09'] == 'OTHER')|(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code010'] == 'OTHER')
                           |(df_Lumped_NonDrug['ICD-10_Claim_Diagnosis_Code011'] == 'OTHER')|(df_Lumped_NonDrug['Primary_Diagnosis_9'] == 'OTHER') 
                           |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code02'] == 'OTHER')
                           |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code03'] == 'OTHER') 
                           |(df_Lumped_NonDrug['ICD-9_Claim_Diagnosis_Code04'] == 'OTHER')]

Report_Lumped_Other  = df_Lumped_Other.rename(columns=lambda col: b.get(str(col)) if str(col) in b else col)

Report_Lumped_Other[['#EOCs','EOC_with_no_drug','#Units','EOC_Total_Pmt_Code_Portion',
         'EOC_Total_Calculated_Allow_Code_Portion',
         ]]

Report_Lumped_Other= Report_Lumped_Other.loc[Report_Lumped_Other['EOC_Total_Pmt_Code_Portion']!=0]

Report_Lumped_Other['#EOCs'].nunique()

Report_Lumped_Other.agg(['sum'])

In [ ]:
from glob import glob
#csv file name and path
file_name = 'All_Insurances'
file_path = os.path.join(output_folder, file_name)
file_path+='.csv'
with open(file_path, 'a') as singleFile:
    for csvFile in glob('../EOC_Raw_Data/EOC_Reports_All_Insurance/Re*.csv'):
        for line in open(csvFile, 'r'):
            singleFile.write(line)